# Cookbook

In this notebook are the basics explained for making questions with widgets. The first part gives an overview of the questions. The second part elaborates on how these questions are built and gives insight into the encountered problems, the choices that are made, and what alternatives are available. The third section shows results from the previous studies, done with IPY widgets and also uses dictionaries to set up questions. If you have any questions, feel free to contact C.D.Hoogervorst@tudelft.nl.

## Cookbook essence

This section gives the short and sweet codes that can be used for making questions that use widgets. Similar questions can be made by changing the content of the question (the question, choices, answer, feedback, etc.) and storing them in the list. Understanding the structure and the characteristics of components is key for making situational layouts. the second part of this cookbook gives here more detailed insight. <br>

The examples start simple, to the end are some additional options like animations and graphs that draws line and dots with mouse input.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import panel as pn
pn.extension("ipywidgets", 'katex', 'mathjax')
import ipywidgets as ipw
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import MultipleLocator
from matplotlib.figure import Figure

from random import shuffle, uniform

from scipy.interpolate import CubicSpline

import sys
from inspect import signature

### Short introduction

Here is a short explanation before diving into the code.

#### Widgets

Widgets are used to give input to a code. Values etc are now provided by changing the value of a slider, checking a box, or pressing on a graph rather than changing the values in the code and then running it. The benefit is that the impacts of changing parameters can be displayed immediately and inputs can be checked as in a question. Here a (submit) button comes in, to initiate checking the input with the exact answer.

"import panel as pn" imports the widgets that are used, Over here you can find the [component gallery](https://panel.holoviz.org/reference/index.html) that shows and explains the various widgets that are used. <br>
- pn.widgets.StaticText (etc.): is used for having text, to ask questions, give feedback, etc. <br>
- pn.widgets.Select, pn.widgets.Checkbox, pn.widgets.FloatInput, etc. are used as input for students to answer questions <br>
- pn.widgets.Button: is used for a button to go through the given answers <br>
- feedback_widget = pn.widgets.TextInput is designed as a text input but is now used as a way to give feedback to students. The feedback is provided through feedback_widget.value = ... <br>

#### The structure of the questions

Functions are made that can be called to show the function. These functions can be put into a package that can be installed with pip and then imported. Jupiter Book also gives the opportunity to show/hide specific cells. <br>

Each function/question has the following structure, where only the first 2 dots are question-dependent:
- A part that builds the questions, and then asks the question to students.
  - Random numbers can be generated and then included in the text.
- A part that defines the answers, and calculates them in numerical questions.
  - This is done for each of the (sub) questions
  - Then all the information is stored in various lists, one for questions, one for inputs of students, one for answers, one for feedback, etc.
- Then the widgets are built by looping over the lists. 
- The widgets are structured and displayed (The code can also end with this)
- The functionality submit button (or other reactive parts) is defined.
  - the submit_button.on_click(function) intiates a function
  - the .on_click might run once when running the code the first time, therefore a nested function is made def button_callback(b), where b stands for the press-button-event.
  - The function that is reached (between brackets) should be above the submit_button.on_click.

#### Randomize questions

An important aspect of the questions is to generate random numbers or randomize the order of the questions/options.
These aspects are considered in the code below, but to highlight them:

- shuffle() is an option to randomize the order of what is insight it. <br>
- uniform(a, b), provides a random number from a to b, excluding the value of b. It has many decimal numbers, so it is usually in the round(number, decimals) function, that defines the number of decimals. This results into x = round(uniform(a,b), decimals)) <br>
- np.random.choice() picks a random value from a list. If you would like a random value from 10 to and including 20 with steps of 5, this can be done by: <br> x = np.arange(10, 20 + 5, 5) and then <br> x = np.random.choice(x). <br>

#### Working with lists

Lists are used many times, not only to store widgets but also to calculate a range of values, for example, the change along a cross-section. List comprehension is a powerful and concise way to make these calculations, in contrast to "for loops" that are commonly used. Many interesting applications can be found [here](https://www.w3schools.com/python/python_lists_comprehension.asp). One example is shown below.

In [ ]:
def func(T):
    return T**2

# the values of T that are considered
T_range = np.arange(0,5+0.25,0.25)

# the y-values for each T
y = [func(T) for T in T_range]

# and plot in a small figure
plt.figure(figsize = (2,1))
plt.plot(T_range,y);

#### Working with many variables

Here is a short introduction to working with many variables in functions, the second part of this cookbook elaborates on this. One benefit is that not all the parameters have to be written as argument when the function is defined and when this function is called.

In [ ]:
# general code, that has to be defined once
class class_variables:
    def __setattr__(self, key, value):
        object.__setattr__(self, key, value)

def classify_variables(params, params2 = {}, max_size_MB = 0):
    max_size = max_size_MB * 1024*1024
    FV = class_variables()
    for key, value in {**params,**params2}.items():
        if sys.getsizeof(value) < max_size or max_size <= 0:
            FV.__setattr__(key, value)
    return FV

# The implementation of storing the variables as classes inside functions

def nested_function(FV,GV, AV):
    print('Value of local parameter:', FV.local_parameter)
    print('Value of global parameter', GV.global_parameter)
    print('Value of global parameter', AV.global_parameter)

def function():
    local_parameter = 2

    # Load and store all the global variables, globals() (can also be inside the nested function)
    GV = classify_variables(globals())

    # Load and store all the function variables, the locals()
    FV = classify_variables(locals())

    # Load and store all variables, the locals and globals.
    # In the other code is this called FV.
    AV = classify_variables(locals(), globals())
    
    nested_function(FV, GV, AV)

global_parameter = 5

function()

### Single multiple choice question

The code below gives one single multiple-choice question. It is the most basic question in providing the question information, making the widgets, displaying them in a structured way, and setting up a submit button. <br>

The code is divided into a function with the general code (single_multiple_choice) and a function with the question (ask_single_multiple_choice). The benefit is that single_multiple_choice() only has to be coded/imported once, preventing repetitive code. The ask_single_multiple_choice() can be named such that it becomes unique, for example, by referring to a chapter or week the question is in with an id/number. Having the questions stored inside a function prevents all the parameters from being stored in the memory, and duplicate names (so overwriting) are prevented.

In [ ]:
def single_multiple_choice(question, choices,  answer, hint, comment):
    # Make the required widgets
    question_widget = pn.widgets.StaticText(value=question)
    choices_widget = pn.widgets.Select(options=choices, name="")
    submit_button = pn.widgets.Button(name="Check")
    feedback_widget = pn.widgets.TextInput(value="", name="", width=500)

    # Build the structure by aligning the widgets
    submit_row = pn.Row(submit_button, feedback_widget)
    quiz_widget = pn.Column(question_widget, choices_widget, submit_row)
    display(quiz_widget)

    def check_answers(event):
        chosen_answer = choices_widget.value
    
        if chosen_answer == answer:
            feedback_widget.value = comment
        else:
            feedback_widget.value = hint

    # Run the function check_answers when the submit button is pressed
    submit_button.on_click(check_answers)

def ask_single_multiple_choice():
    # The information of the question
    question_1 = "A large continental shelf width is at a:"
    choices_1 = ["Leading edge", "Trailing edge", "Marginal sea"]
    answer_1 = choices_1[1] # 0-based index
    hint_1 = "Unfortunately not, here is a hint ..."
    comment_1 = "Indeed, some additional information..."

    single_multiple_choice(question_1, choices_1, answer_1, hint_1, comment_1)

ask_single_multiple_choice()

### Several combined multiple-choice questions

Several multiple-choice questions can be asked, and then a total score can be provided to the students. Each of the multiple-choice options can be provided with feedback. If no feedback wants to be provided keep the hint or comment parameter empty, as in question 2. Also, the order of questions can be randomized, which is optional. <br> The code in the cell below has to be imported/loaded once. Below that is an example of how the question can be setup.

In [ ]:
def several_multiple_choice(questions, choices, answers, hints, comments, random_order = False, n_questions_max = 1e10):  
    # An empty list to store the widgets
    all_widgets = []  # For visualization, store all the widgets in the order they are going to be displayed
    question_widgets = []  # Store all the question widgets in a list
    choices_widgets = []  # store all the choices widgets in a list
    feedback_widgets = [] # store all the feedback widgets for subquestions

    # Optionally: randomize the order of the questions
    order = np.arange(0, len(questions), 1)
    if random_order == True:
        shuffle(order)

    # Optionally: limit the number of questions shown
    if len(questions) > n_questions_max:
        order = order[0:n_questions_max]
    
    # make the widgets in a loop, one widget states the question and one shows the options that can be selected.
    for i in np.array(order):
        question, choice, answer = questions[i], choices[i], answers[i]
        question_widget = pn.widgets.StaticText(value=question)
        choices_widget = pn.widgets.Select(options=choice, name="")
        feedback_widget =pn.widgets.StaticText(value="", name="")

        # store the widgets on type, make looping through them possible, the order is unaffected by randomizing
        question_widgets.append(question_widget)
        choices_widgets.append(choices_widget)
        feedback_widgets.append(feedback_widget)

        # store the widgets (again) for display purposes. It remains the same objects as in the previous list. 
        allign_question_feedback = pn.Row(choices_widget, feedback_widget)
        
        all_widgets.append(question_widget)
        all_widgets.append(allign_question_feedback)
    
    # make a submit button and a feedback button
    submit_button = pn.widgets.Button(name="Check")
    final_feedback_widget = pn.widgets.TextInput(value="", name="", width = 220)
    
    # Allign the submit button and the feedback widget horizontally
    HBox_check = pn.Row(submit_button, final_feedback_widget)
    all_widgets.append(HBox_check)
    
    # unpacks the list by an asterisk (*)
    quiz_widget = pn.Column(*all_widgets)
    
    # make a function to calculate the score and to give feedback
    def check_answers(event):
        score = 0
    
        for i in range(len(order)):
            answer = choices_widgets[i].value
            correct_answer = answers[i]
        
            if answer == correct_answer:
                score += 1
                feedback_widgets[i].value = comments[i]

            if answer != correct_answer:
                feedback_widgets[i].value = hints[i]
        
        final_feedback_widget.value = "Your score is " + str(score) + "/" + str(len(order))
    
    submit_button.on_click(check_answers)
    display(quiz_widget)

An example of the question itself, this should be in an initalizer or a hidden cell. Question 1 provides the feedback given, in question 2 will be no feedback given. The lists at the bottom can optionally be made through eval() in a loop.

In [ ]:
def ask_several_multiple_choice_feedback():
    # The information of the questions
    question_1 = "A large continental shelf width is at a:"
    choices_1 = ["Leading edge", "Trailing edge", "marginal sea"]
    answer_1 = choices_1[1]
    hint_1 = "Unfortunately not, here is a hint ..."
    comment_1 = "Indeed, .. some additional information ... "
    
    question_2 = "The oceanic geoid is: "
    choices_2 = [
        "An oval shape",
        "The shape of the ocean surface under only gravity forces",
        "A geo triangle with a different shape",
        "The actual ocean surface",
    ]
    answer_2 = choices_2[1]
    hint_2 = ""
    comment_2 = ""
    
    # store the questions in a list
    questions = [question_1, question_2]
    choices = [choices_1, choices_2]
    answers = [answer_1, answer_2]
    hints = [hint_1, hint_2]
    comments = [comment_1, comment_2]

    several_multiple_choice(questions, choices, answers, hints, comments, random_order = True)

ask_several_multiple_choice_feedback()

### Multiple selection

In the code below are correct and false statements presented. Students have to select the check boxes related to correct answers. Students gain points when they choose the correct answers but lose points if they check incorrect answers. The lowest score is 0 points. <br> The code below has to be loaded/imported once, below this is an exemplary question.

In [ ]:
def multiple_selection(question, correct_statements, false_statements):
    # Make an empty list to store the widgets (references), checkboxes, and true/false statements sorted.
    check_boxes = []  # all the boxes to click
    all_statements = []  # all the statements

    # question widget
    question_widget = pn.widgets.StaticText(value=question)

    # For visualization, the maximum length/width of the question, with a minimum width of 200
    max_length = max(len(item) for item in correct_statements + false_statements)
    width_statement = max(250, max_length*7)
    
    # An empty list for visualization to store the HBoxes that contain the widgets, one statement and the corresponding checkbox
    all_widgets = []

    for statement in correct_statements + false_statements:
        add_statement = pn.widgets.StaticText(value=statement, width = width_statement)
        check_box_widget = pn.widgets.Checkbox(value=False, width=120)
        HBox1 = pn.Row(add_statement, check_box_widget)
    
        all_statements.append(add_statement)
        check_boxes.append(check_box_widget)
        all_widgets.append(HBox1)
    
    # randomize the order of statements
    shuffle(all_widgets)
    
    # add submit button and output, which come on the bottom
    submit_button = pn.widgets.Button(name='Check')
    output_widget = pn.widgets.TextInput(value='', placeholder='', disabled=False)
    
    # make an additional HBox for aligning the submit button and the output widget
    HBox2 = pn.Row(submit_button, output_widget)
    all_widgets.append(HBox2)
    
    # align all the HBoxes beneath each other (oldest below if not randomized) and display them.
    quiz_widget = pn.Column(question_widget, *all_widgets)
    
    # Check the checkbox for each statement and calculate the score.
    def check_answers(event):
        score = 0
    
        for i in range(len(check_boxes)):
            check_box = check_boxes[i]
            statement = all_statements[i].value
    
            if statement in correct_statements:
                if check_box.value == True:
                    score += 1
                else:
                    score -= 0
    
            if statement not in correct_statements:
                if check_box.value == True:
                    score -= 1
                else:
                    score -= 0
    
        score = np.max([score, 0])
        output_widget.value = f'You have: {round(score/len(correct_statements)*100,0)}% of the points'
    
    submit_button.on_click(check_answers)
    return quiz_widget # This is similar to .serve(), it can also be done through: display(quiz_widget)

The function below is the question inside the notebook.

In [ ]:
def ask_multiple_selection():
    question = 'Which statements are correct/positive?'
    correct_statements = ["Correct", "The earth is round", "Good"]
    false_statements = ["False", "The earth is a cube"]

    return multiple_selection(question, correct_statements, false_statements)


ask_multiple_selection()

### Select correct statements

For this question do students have to select the correct statement out of multiple options. They gain a final score that indicates the number of correct answers. (Losing points with a wrong answer can be added as well). The order of the questions is randomized. The code in the cell below has to imported/loaded once.

In [ ]:
def select_statement(questions, answers, correct_answers_id):
    # define the widgets for visualization, make for each a row with a question and answer
    # Store all the rows with statements in a list for visualization, and in toggle_widgets for checking the answer
    Rows = []
    toggle_widgets = []
    for i in range(len(correct_answers_id)):
        question_widget = pn.widgets.StaticText(value=questions[i], width = 150)  #statement
        
        radio_group_widget = pn.widgets.RadioButtonGroup(name='Radio Button Group', options=answers, button_type='default')
        toggle_widgets.append(radio_group_widget)
        
        add_row = pn.Row(question_widget, radio_group_widget)
        Rows.append(add_row)

    # randomize the order of statements
    shuffle(Rows)

    # Add a submit button with a feedback option next to it
    submit_button =  pn.widgets.Button(name="Check")
    feedback_widget = pn.widgets.StaticText(value="", name="", width=500)
    submit_row = pn.Row(submit_button, feedback_widget)
    
    # Structure the widgets
    UI = *Rows, submit_row

    # check the answer and give feedback
    def check_answers(button):
        score = 0

        for i in range(len(correct_answers_id)):
            if toggle_widgets[i].value == answers[correct_answers_id[i]]:
                score += 1

        # print(toggle_widget.value)
        feedback_widget.value = "Your score is " + str(score) + "/" + str(len(correct_answers_id))

    submit_button.on_click(check_answers)

    return UI

And the question can be asked in this way:

In [ ]:
def ask_select_statement():

    # define the questions and store them in a list
    Q1 = 'Question 1'
    Q2 = 'Question 2'
    Q3 = 'Question 3'
    questions = [Q1, Q2, Q3]

    # The possible answers
    answers = ["Shallow", "Intermediate", "Deep"]

    # The correct answers, 0-based id, can be calculated with functions when working with randomized numbers
    correct_answers_id = [0,1,2]

    # Add the text of the question on top
    text_general = "Select if the wave described on the right experiences shallow, intermediate, or deep water."

    # make the widgets and display them
    text_widget = pn.widgets.StaticText(value=text_general)
    question_widgets = select_statement(questions, answers, correct_answers_id)
    display(text_widget, *question_widgets)

ask_select_statement()

### Select/slide figures

make a slider in which students can use a slider to select a figure. How path_figures can be defined using Git Hub have to be coordinated. The code is now as raw text to prevent it from running and giving an error notification that the figures are not found. An improved version will be made when these options have been streamlined.

The figures and code that displays them through widgets was made by Mario van den Berg, for the test notebook about the escofier curves.

### Numerical question

The numerical question should consider various aspects. <br>
- It should check the answer, where students can give more decimal numbers than requested. The question should give feedback, optionally when the answer is wrong and/or correct.
When 3 attempts are made the correct answer is given.
- The function Q1() defines the questions, feedback, etc. It now shows 3 times the same question, which can be changed by changing the lists of questions, units, answers, FB_good, and FB_wrong.<br>
- The function nummeric_question_body() is the core of the layout.  This can be changed if a different structure of widgets is preferred.

Note that functions can have a lot of input parameters (So the check_nummeric_answers can quickly require many more parameters). It is possible to prevent defining all the widgets in all the functions by storing all the local parameters in one parameter that is passed on to the function. The value of the parameters can then be extracted from this parameter. The second part of this cookbook (Full recipe) elaborates on this.
<br>

The lists can become long when many (sub)questions are asked. Working with eval() in a loop can be a way to work with this, no example is added in this cookbook (yet), you can look in week_2_initialize under W2_Q7_questions().

In [ ]:
def limit_answer(answer, f_margin = 0):
    # if no margin is given, adding additional decimals are counted correctly.
    # if a margin if given, the answer should be within a range from the correct answer.
    # The maximal deviation is determined with the factor (f_margin)
    s = str(answer)

    if f_margin == 0: 
        # inspired by: https://stackoverflow.com/questions/35585950/find-the-number-of-digits-after-the-decimal-point
        if not '.' in s:
            n_decimal = 0
        else:
            n_decimal = len(s) - s.index('.') - 1
    
        range = 5*10**-(n_decimal+1)

    if f_margin != 0:
        range = f_margin * answer

    return range

def check_nummeric_answers(id, answer, unit, FB_G, FB_W, num_widget, feedback_widget, attempt, f_margin = 0):
    
    def button_callback(b):
        attempt.value += 1
        #print("debug question:", id, 'attempt', attempt.value ,', response:' ,num_widget.value, ', answer:',answer)

        # the answer is within the boundaries, print positive feedback
        if np.abs(answer - num_widget.value) < limit_answer(answer, f_margin):
            if len(FB_G) != 0:
                feedback_widget.value = FB_G
            else:
                feedback_widget.value = 'Well done, this is correct!'

        # the answer is NOT within boundaries, provide feedback based on the number of attempts
        if np.abs(answer - num_widget.value) >= limit_answer(answer, f_margin):

            if attempt.value < 3 and len(FB_W) > 0:
                feedback_widget.value = FB_W
                
            if attempt.value < 3 and len(FB_W) == 0:
                feedback_widget.value = 'Oops, there seems to be a mistake'
                
            if attempt.value >= 3:
                feedback_widget.value = 'The correct answer is ' + str(answer) + str(unit) + '.'

    return button_callback  # otherwise gives TypeError: 'NoneType' object is not callable

def nummeric_question_body(questions, units, answers, FB_good, FB_wrong, random_order = False, f_margin = 0):
    all_widgets = []
    attempts = []

    order = np.arange(0, len(questions), 1)
    if random_order == True:
        shuffle(order)

    for i in np.array(order):
        question, unit, answer, Q_FB_G, Q_FB_W = questions[i], units[i], answers[i], FB_good[i], FB_wrong[i]
        id = i+1 
        question_widget = pn.widgets.StaticText(value=question, width = 750)
        unit_widget = pn.widgets.StaticText(value=unit, width = 40)
        num_widget = pn.widgets.FloatInput(value=0, step=0.01, width = 100)
        #feedback_widget = pn.widgets.TextInput(value="", name="", width=500)
        feedback_widget = pn.widgets.StaticText(value="", name="", width=500)
        submit_button =  pn.widgets.Button(name="Check")
        
        Hbox = pn.Row(num_widget, unit_widget, submit_button, feedback_widget)       
        quiz_widget = pn.Column(question_widget, Hbox)

        all_widgets.append(quiz_widget)

        # the values for the submit button are determined at the moment these are created.
        attempt = pn.widgets.FloatInput(value=0)
        attempts.append(attempt)
        submit_button.on_click(check_nummeric_answers(id, answer, unit, Q_FB_G, Q_FB_W, num_widget, feedback_widget, attempt, f_margin))
        
    return all_widgets

def Q1():
    T1 = round(uniform(5, 8), 1)
    h1 = round(uniform(0.5, 5), 1)

    text_general = "Can you asses the wave length in three different ways? Firstly through an iterative approach. The wave period (T) is " + str(T1) + " seconds, and the water depth (h) is " + str(h1) + " m?"
    text_widget = pn.widgets.StaticText(value=text_general, width = 750)
    
    Q1_text = "Q1a) What is the deep water wavelength?"
    Q1_unit = " m"
    L = 9.81 * T1**2 / (2 * np.pi)
    Q1_answer = round(L, 2)
    Q1_FB_G = 'Indeed, the deep water wavelength is in this way related to the wave period.'
    Q1_FB_W = 'There is a mistake, the only variable is the wave period.'

    questions = [Q1_text, Q1_text,Q1_text]
    units = [Q1_unit, Q1_unit, Q1_unit]
    answers =[Q1_answer, Q1_answer, Q1_answer]
    FB_good = [Q1_FB_G, Q1_FB_G, Q1_FB_G]
    FB_wrong = [Q1_FB_W, Q1_FB_W, Q1_FB_W]

    
    all_widgets = nummeric_question_body(questions, units, answers, FB_good, FB_wrong, random_order = True)
    
    display(pn.Column(text_widget,*all_widgets))
    
Q1()

### Update graphs with sliders

Plots can be redrawn when parameters are changed. This can be done with the [Param component](https://panel.holoviz.org/reference/panes/Param.html) of panel. The coding is quite difficult, while it is easy with the interact function of IPY widgets. The application of the interact function is shown below. In the future, this might be replaced with another tool. This transition with questions is probably relatively easy since the majority of the code is related to the content of the questions. One downside is that is difficult to hide the automatically generated output (<function __main__.plot_slider_graph(a, L)>)<br>

The structure is relatively simple. The (slider) widgets are defined in a function, while the other parts (the calculation, plotting, and optionally questioning) are inside a function that is called with interact().

In [ ]:
from ipywidgets import interact

def plot_slider_graph(a, L):
    fig, axs = plt.subplots(nrows = 1, ncols = 1, figsize = (9,6), sharex=True, sharey = False)
    fig.subplots_adjust(hspace=0.05)
    fig.subplots_adjust(wspace=0.04)
                        
    x = np.linspace(0,12,100)
    
    eta = a * np.sin(2 * np.pi / L * x)
    axs.plot(x, eta)

def slider_graph():
    # Create interactive widgets, requires IPY Widgets, widgets from panel do not work
    #L = pn.widgets.FloatSlider(name='Float Slider', start=0, end=3.141, step=0.01, value=1.57)
    a = ipw.FloatSlider(value=4, min=0.01, max=50, step=0.01, description="a [s]")
    L = ipw.FloatSlider(value=6, min=0.01, max=50, step=0.01, description="L [s]")

    # Use the interactive function to update the plot
    Plot = interact(plot_slider_graph, a=a, L=L);
    display(Plot)

slider_graph()

### Updating graphs with (submit) buttons

Plots can be redrawn when parameters are changed. This can be done with the [Param component](https://panel.holoviz.org/reference/panes/Param.html) of panel. The code below shows the implementation of having a graph updated when a (submit) button is pressed. The graphs are added similarly to other widgets, now with additional attention to defining the axis of the graph (ax), in which the figure is displayed (fig), and the panes widget (pane). The code below shows them for making two graphs. 

It is not recommended to have calculations etc. It is hard to return values. The simplest way (or workaround) is by using widgets and changing the value. Below is the counter used as input to change the value of the graph each time the button is pressed. Further on in this notebook is a section related to working with parameters in functions.

In [ ]:
#set plot settings and make plots   
titles = ['plot1', 'graph2']
figures = []
plots = []
panes = []
for title in titles:
    fig = Figure((5,2.5))
    ax = fig.subplots()
    pane = pn.pane.Matplotlib(fig, dpi=96)

     # Add some extra space for labels at the axis
    fig.subplots_adjust(bottom=0.25) 
    fig.subplots_adjust(left=0.2)

    # store everything in a list
    figures.append(fig)
    plots.append(ax)
    panes.append(pane)

# set a function that defines what has to be done when the button is pressed
def plot_graph(figures, plots, panes, start_value_right, counter):

    def button_callback(b):
        ax = plots[0]
        ax.clear() # remove previous lines
        ax.plot(np.linspace(0,10,10), np.linspace(0,start_value_right + counter.value ,10), label = 'L [m]')
        ax.set_xlabel("x-label")
        panes[0].object = figures[0]

        ax = plots[1]
        ax.clear() # remove previous lines
        ax.plot(np.linspace(0,10,10), np.linspace(0,start_value_right - counter.value ,10), label = 'L [m]')
        ax.set_xlabel("x-label")
        panes[1].object = figures[1]

        counter.value += 1

    return button_callback  # otherwise gives TypeError: 'NoneType' object is not callable

start_value_right = 1

counter = pn.widgets.FloatInput(value=0)

submit_button =  pn.widgets.Button(name="Update graph")
submit_button.on_click(plot_graph(figures, plots, panes, start_value_right , counter))

pn.Column(submit_button, *panes)

### Coding question - Check value of parameter(s)

Students can be asked to compute values of specific parameters in a code field. The students can then load their result into a widget, which has the option to check their results with the answers and gives feedback. It gives a notification if one of the parameters are not defined when loading. It also gives the correct answer after 3 attempts, When the widget does not have to be runned to update the values, which is the case in notebooks.

Below is the code that has to be defined/imported only once.

In [ ]:
# general code, that has to be defined once
class class_variables:
    def __setattr__(self, key, value):
        object.__setattr__(self, key, value)

def classify_variables(locals, max_size_MB = 0):
    max_size = max_size_MB * 1024*1024
    FV = class_variables()
    for key, value in  {**globals(), **locals}.items():
        if sys.getsizeof(value) < max_size or max_size <= 0:
            FV.__setattr__(key, value)
    return FV

def add_local_variables_to(FV,locals, max_size_MB = 0):
    max_size = max_size_MB * 1024*1024
    for key, value in  locals.items():
        if sys.getsizeof(value) < max_size or max_size <= 0:
            FV.__setattr__(key, value)
    return FV

def check_code_values(IV):
    # The function variable (FV) is now defined as Input Variable (IV)
    # The input variable and the newly defined local variable will be merged to FV

    
    def get_coded_values(FV,GV):
        def button_callback(b):
            try:
                for i, param in enumerate(FV.check_parameters):
    
                    # get the value that students gave
                    response = getattr(GV, param)
    
                    #store them in the widget
                    FV.all_parameter_widgets[i].value = response
                    
                FV.debug_widget.value = ''
            except:
                FV.debug_widget.value = '<b> Careful, not all parameters are defined! </b>'
    
        return button_callback

    def check_coded_values(FV,GV):
        def button_callback(b):
            FV.attempt.value += 1
            
            for i, param in enumerate(FV.check_parameters):
                #response = getattr(GV, param)
                response = FV.all_parameter_widgets[i].value
                answer = getattr(FV, param)
    
                if np.abs(response - answer) < FV.f_margin * answer:
                    FV.all_feedback_widgets[i].value = 'Nice, this is good!'
    
                if np.abs(response - answer) >= FV.f_margin * answer:
                    if FV.attempt.value < 3:
                        FV.all_feedback_widgets[i].value = 'This one is incorrect, try again!'
                    if FV.attempt.value >= 3:
                        FV.all_feedback_widgets[i].value = 'This one is incorrect, the answer should be ' + str(answer) + '.'
                
                #print('Debug: ', param, response, answer)
    
        return button_callback

    all_parameter_widgets = []
    info_widgets = []
    all_feedback_widgets = []
    for name, param in zip(IV.name_parameters, IV.check_parameters):
        symbol_widget = pn.widgets.StaticText(name='', value= name, width = 100)
        parameter_widget = pn.widgets.FloatInput(name='', value=0, width = 100)
        feedback_widget = pn.widgets.StaticText(value='')
        
        all_parameter_widgets.append(parameter_widget)
        all_feedback_widgets.append(feedback_widget)
        
        new_row = pn.Row(symbol_widget, parameter_widget, feedback_widget)
        info_widgets.append(new_row)

    debug_widget = pn.widgets.StaticText(value='')
                                 
    get_values_button =  pn.widgets.Button(name="Load values")
    check_values_button =  pn.widgets.Button(name="Check loaded values")

    GV = classify_variables(globals())
    FV = add_local_variables_to(IV,locals())

    # The error margin is set at 0.01% if it is not defined
    if 'f_margin' not in FV.__dict__:
        FV.__setattr__('f_margin', 0.0001)
    
    get_values_button.on_click(get_coded_values(FV, GV))
    check_values_button.on_click(check_coded_values(FV, GV))
    
    row_buttons = pn.Row(get_values_button, check_values_button, debug_widget)
    Input_col = pn.Column(row_buttons, *info_widgets)
    display(Input_col)

The first function is used to ask the question to students and define the values of the parameters. And the second is used to check the answer. Note that the names of the functions should be unique.

In [ ]:
def Question():
    # the question-related parameters
    L1 = 6
    L2 = round(uniform(7,10), 1)

    # The question that is asked
    question = 'What is k1 and k2 when the lengts of wave 1 and 2 are ' + str(L1) + ' and ' + str(L2) + ' meter? Complete and run the code below and you can check your results.'
    
    # make the attempt counter, one for each subquestion
    attempt = pn.widgets.FloatInput(value=0)

    # Required widgets for functionality, should not be changed
    question_widget = pn.widgets.StaticText(name='', value= question)
    display(question_widget)

    # define a new global variable, with a unique name (This is related to week 2, question 9)
    global W2_Q9_param 
    # store the question-related parameters and the widget 'attempt' 
    # These parameters can also be stored as classes
    W2_Q9_param = L1, L2, attempt

def Check_Question():
    L1, L2, attempt = W2_Q9_param
    
    # define the parameter names that have to be checked
    check_parameters = ['k1', 'k2']

    # define the names of the parameters as they are displayed
    name_parameters = ['k 1', 'k 2']

    # define the answers for the questions, which are described in check_parameters
    k1 = 2*np.pi/L1
    k2 = 2*np.pi/L2

    # a standard part that stores the parameters and displays the UI
    # f_margin = 0, used to define the maximum error, which a a factor to the answer, default 0.0001 = 0.01%
    FV = classify_variables(locals())
    check_code_values(FV)

The question in the Jupiter (note)book can be loaded through:

In [ ]:
Question()

A structure of the code has to be given, at least the names of the parameters. The parts that students have to fill in can be indicated by 3 dots or red text, as in:

In [ ]:
k1 = ...
k2 = 'fill this in'

Check_Question()

One possible answer of students can be:

In [ ]:
L1 = 6
k1 = 2*np.pi/L1*0.9999
k2 = 2

Check_Question()

### Coding question - Check a function and make a graph

The function below loads the function the students make. It will plot this function if this is possible otherwise, it will give a warning. 
The check_code_function() should be loaded/imported once, no changes have to be made.

In the second part of this cookbook (Full recipe) are examples of how subplots can be applied, where each subplot can check one or multiple functions, and how a returned range/array can be checked. 

The second part also gives suggestions on how these functions can be adjusted, so that they are more flexible and can be structured in various ways.

In [ ]:
def check_code_function(fig, horizontal_axis, function_name, correct_function, par_x_axis, f_margin = 0.001):
    # The error margin (f_margin) is set at 0.1% if it is not defined

    # make the graph and show the correct answer
    ax = fig.subplots()
    pane = pn.pane.Matplotlib(fig, dpi=100)
    
    #  Plot the answer if the students function is found
    try:
        # Load the student-made function from globals
        function = globals()[function_name]

        # Use the student made function to calculate their answer
        # Add the arguments to the name of the function, for eval()
        # + Replace the argument on the x-axis so that it can be assessed with list comprehension.
        # https://docs.python.org/3/library/inspect.html
        # https://peps.python.org/pep-0362/    
        sig = signature(function)
        sig_function = str(sig).replace(par_x_axis, 'par_x_axis')
        function = function_name + str(sig_function)
        student_answer = [eval(function) for par_x_axis in horizontal_axis]

        # get the function for the correct answer
        sig = signature(correct_function)    
        sig_function = str(sig).replace(par_x_axis, 'par_x_axis')
        correct_function2 = str(correct_function.__name__) + sig_function

        #calculate the correct answer
        correct_answer = []
        for par_x_axis in horizontal_axis:
            correct_answer.append(eval(correct_function2))
 
        # plot the answers
        line = ax.plot(student_answer, label = 'Your answer', zorder = 1)
        line2 = ax.plot(correct_answer, label = 'Correct answer', zorder = 1)

        # check if the answer is correct and plot it before/below lines
        changes = np.array(student_answer) - np.array(correct_answer)
        inaccuracy = np.abs(1-np.array(correct_answer)/np.array(student_answer))

        # Plot comment if the answer is correct
        y_loc = (np.nanmean(correct_answer) + np.nanmin(correct_answer))/2
        x_loc = np.mean(ax.get_xticks())
        if np.nanmax(changes) == 0:      
            text = ax.text(x_loc, y_loc, 'Perfect!', fontsize=12, color = '#1b5a00', ha='center', va='center', zorder = 0)

        if np.nanmax(changes) != 0 and np.nanmax(inaccuracy) < f_margin:
            text = ax.text(x_loc, y_loc, 'Good!', fontsize=12, color = '#1b5a00', ha='center', va='center', zorder = 0)

        # set title and legend
        title = function_name + str(sig)
        title = title.replace('_', ' ')
        ax.set_title(title)
        ax.legend()
           
    except:
        text_failed = 'Almost there, \n your function can not be plotted, \n please try to fix the bug'
        x_ticks = ax.get_xticks()
        y_ticks = ax.get_yticks()
        text = ax.text(np.average(x_ticks),np.average(y_ticks), text_failed, fontsize=16, color = 'r', ha='center', va='center')

    # update the graph
    display(pane)

This is the code related to building one question, and will be loaded/imported into the notebook <br>
This question/function defines:
- the name of the function is that students make
- the name of the variable that is used on the x-axis
- the correct function
- the maximum allowable error margin (is a factor)
- the size of the figure

In [ ]:
def Show_question_1():
    # define the name of the function that the students will make
    function_name = "student_function"

    # define the name of the parameter plotted on the horizontal axis
    parameter_x_axis = 'x'

    # set the horizontal axis of the graph
    horizontal_axis = np.arange(0,10+1,1)

    # define the correct function
    def correct_function(a,b,x):
        y = a*x + b
        return y

    # set the acceptable computational error (ratio)
    f_margin = 0.001 # 0.001 = 0.01%

    # set the size of the figure
    fig = Figure((5,2.5))

    # call the function that builds the backend.
    check_code_function(fig, horizontal_axis, function_name, correct_function, parameter_x_axis, f_margin)

The cell below shows an example of how the question can be asked in the notebook that is provided to students. 

In [ ]:
def student_function(x,a):
    y = 'fill this secion in'
    return y
    
Show_question_1()

Students should define the values of the parameters en the function, which can include new parameters/arguments. The correct funcion should be y=ax+b.

In [ ]:
a,b,c = 1,2,0.5
x = 3
def student_function(a,b,x,c):
    y = a*x + b*2 - c*x
    return y

Show_question_1()

It will give positive response when the function is correct, it will give perfect when the error = 0, and good when the error is within the given margin. The deviation can be due to a different number of iterations, for example.

In [ ]:
def student_function(a,b,x):
    y = a*x + b
    return y

Show_question_1()

### Animation graph

The code below gives the option to run an animation that shows harmonic wave components propagating over time.

In [ ]:
# set input parameters (can be made with widgets)
%matplotlib widget
a = 1
w = 0.35
k = 0.25
delta_t = 30 # ms

# Set initial conditions
t = 0
x = np.linspace(0, 100, 1000)  # m
eta = a * np.sin(w * t - k * x)

# Create figure, axes, and initial values
fig, ax = plt.subplots()
line, = ax.plot(x, eta)

def update_line(change):
    t = change.new
    eta = a * np.sin(w * t - k * x)
    line.set_ydata(eta)
    fig.canvas.draw()
    #print(t)

discrete_player = pn.widgets.DiscretePlayer(name='Discrete Player', options=np.arange(0,500,delta_t/1000).tolist(), value=0, loop_policy='loop', interval = delta_t)
discrete_player.param.watch(update_line, 'value')

pn.panel(discrete_player)

### Animation graph with widgets

Animated graphs with widgets can use widget input. The main difference is that the widget.value is used rather than only the parameter. In the code below is a widget used for parameter a. To get the computation working, a.value is used rather than a. Note that the boundaries of the graph are not automatically adjusted in this manner.

In [ ]:
# set input parameters (can be made with widgets)
%matplotlib widget
a = pn.widgets.FloatInput(name='', value=1, width = 100)
w = 0.35
k = 0.25
delta_t = 30 # ms

# Set initial conditions
t = 0
x = np.linspace(0, 100, 1000)  # m
eta = a.value * np.sin(w * t - k * x)

# Create figure, axes, and initial values
fig, ax = plt.subplots()
line, = ax.plot(x, eta)

def update_line(change):
    t = change.new
    eta = a.value * np.sin(w * t - k * x)
    line.set_ydata(eta)
    fig.canvas.draw()

discrete_player = pn.widgets.DiscretePlayer(name='Discrete Player', options=np.arange(0,500,delta_t/1000).tolist(), value=0, loop_policy='loop', interval = delta_t)
discrete_player.param.watch(update_line, 'value')

display(a)
pn.panel(discrete_player)

### Draw on a graph

The code below gives the option to make a line that passes through the drawn points, using cubic splines. In this way students can be asked to draw a graph. The relative values, gradients, or areas underneath the line can be assessed to give a score to the student.

In [ ]:
%matplotlib widget

# list to store x and y coordinates and the lines
points_x = [0,10]
points_y = [1,1]

# list of drawn objects
lines = []

def mouse_click(event):

    
    if event.button == 1:  # 1 Left mouse button, 3 right mouse button
        points_x.append(event.xdata)
        points_y.append(event.ydata)
        
        scatter1 = ax.scatter(points_x, points_y, c='k', s=40)
        scatter2 = ax.scatter(points_x, points_y, c='w', s=20)
              
        if len(points_x) > 1:
            
            # group the points
            coords = zip(points_x, points_y)

            # sort the points on x-coordinate
            sorted_pairs = sorted(coords, key=lambda parameter: parameter[0])
            sorted_x, sorted_y = zip(*sorted_pairs)
            
            # define cubic spline interpolation
            f = CubicSpline(sorted_x, sorted_y, bc_type='natural')
            x_new = np.linspace(0, 10, 1000)
            y_new = f(x_new)
                       
            # remove previous lines
            for line in lines:
                line.remove()
            lines.clear()
                        
            # plot the new line
            line, = ax.plot(x_new,y_new, linestyle='-', color = 'k')
            lines.append(line)
            
            
    if event.button == 3:  # 1 Left mouse button, 3 right mouse button
        
        # get mouse location
        mouse_x = event.xdata
        mouse_y = event.ydata

        # get the distances from all the points to the mouse in a list
        distance = []
        for x,y in zip(points_x, points_y):
            distance.append( ((x-mouse_x)**2 + (y-mouse_y)**2)**0.5)

        # get the id of the point nearest to the mouse
        id_low = distance.index(min(distance))
        
        if id_low != 0:# and id_low != (len(points_x)):

            # delete the closest point
            del(points_x[id_low])
            del(points_y[id_low])
        
            # regroup the points
            coords = zip(points_x, points_y)

            # sort the points on x-coordinate
            sorted_pairs = sorted(coords, key=lambda parameter: parameter[0])
            sorted_x, sorted_y = zip(*sorted_pairs)

            # define new cubic spline interpolation
            f = CubicSpline(sorted_x, sorted_y, bc_type='natural')
            x_new = np.linspace(0, 10, 100)
            y_new = f(x_new)

            # Clear the plot and reset axis settings
            ax.clear()  
            ax.set_xlim(0, 10)
            ax.set_ylim(0, 3)
            ax.set_xlabel('Alongshore location (x) [m]')
            ax.set_ylabel('sediment transport rate magnitude')

            ## remove previous lines
            for line in lines:
                line.remove()
            lines.clear()

            # plot the remaining dots and new spline
            scatter1 = ax.scatter(points_x, points_y, c='k', s=40)
            scatter2 = ax.scatter(points_x, points_y, c='w', s=20)

            line, = ax.plot(x_new,y_new, linestyle='-', color = 'k')
            lines.append(line)

            fig.canvas.draw()

# Create figure
fig, ax = plt.subplots()
fig.canvas.toolbar_visible = False # dont show toolbar
ax.set_xlim(0, 10)
#ax.set_ylim(0, 3)

# make initial plot
scatter1 = ax.scatter(points_x, points_y, c='k', s=40)
scatter2 = ax.scatter(points_x, points_y, c='w', s=20)

line, = ax.plot(points_x,points_y, linestyle='-', color = 'k')
lines.append(line)

# activate the function mouse_click when a button is pressed
fig.canvas.mpl_connect('button_press_event', mouse_click);

plt.xlabel('Alongshore location (x) [m]')
plt.ylabel('sediment transport rate magnitude')

plt.show();

### Select on graph

Interaction with a graph can be done in many more ways. Students can be asked to select (click on) a part of a figure that shows a diagram. Also, multiple options can be checked simultaneously by moving coloured 'circles' to their correct spots, for example. These question types have not been developed, yet.

## Full recipe

In this section of the notebook are the basics for various question types shown. The guide starts with the implementation of making widgets in setting up questions. After that are functions used, to gain additional functionalities and clearer code. The end result is added in the first part of this cookbook. 

More information on the implementation of IPY Widgets in panel can be found at:<br>
https://panel.holoviz.org/reference/panes/IPyWidget.html

All the possible widgets are explained at:<br>
https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html <br>
https://panel.holoviz.org/reference/index.html#widgets <br>
Combining these with other utilities in panel gives the possibility to ask a large variety of questions to students.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import panel as pn
pn.extension("ipywidgets", 'katex')
import ipywidgets as ipw
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import MultipleLocator

from random import shuffle, uniform

### Making widgets

Before diving in how to the questions types are made, the widgets that are used are from [the component gallery](https://panel.holoviz.org/reference/index.html). The site shows the various types of widgets that can be made and how these can be organized. And comes in princple down to:

In [ ]:
a1 = pn.widgets.FloatInput(name='a1 [m]', start=0, end=3, step=0.01, value=1, width = 75)
a2 = pn.widgets.FloatInput(name='a2 [m]', start=0, end=3, step=0.01, value=1, width = 75)

column = pn.Column(a1,a2)

display(column)# serve might be a better option

The function below can be used to add a description on the left and right side, rather than on top, which might give a cleaner view (as it was with IPY widgets before). A possible implementation can be on the left side a description, like a parameter name, and on the right side the units. This new_widget function is not (yet) implemented in the other cookbook functions.

In [ ]:
def new_widget(TYPE, **kwargs):
    # make a new widget with text on the left and right side
    # Arguments/setting for the left, center, and right widgets can be defined
    # true a dictionary in Args_L={'value': 1}, Args_C={...}, and Args_R={...} respectively
    
    # if width is not in **kwargs, add width = 100
    kwargs['Args_L']['width'] = kwargs['Args_L'].get('width', 100)
    kwargs['Args_C']['width'] = kwargs['Args_C'].get('width', 100)
    kwargs['Args_R']['width'] = kwargs['Args_R'].get('width', 100)

    left_widget = pn.widgets.StaticText(**kwargs.get('Args_L'))
    center_widget = getattr(pn.widgets, TYPE)(**kwargs.get('Args_C'))
    right_widget = pn.widgets.StaticText(**kwargs.get('Args_R'))
    
    return pn.Row(left_widget, center_widget, right_widget)

widget = new_widget('FloatInput', Args_C={'value': 1}, Args_L={'value': 'Text on the left side', 'width': 100}, Args_R={'value': 'Text on the right side', 'width': 120})
display(widget)

The value of the widget can be retrieved by:
Note that you can change the value inside the widget and run the cell below again.

In [ ]:
widget[1].value

### Multiple choice

Here is an example of a multiple-choice selection question. It does not focus on the layout of the widgets. The layout can further be improved by aligning the question_widget and choices_widget (and submit button and feedback widget) through one HBox and by setting the width and optionally colors etc of the widgets.

#### Single question

Give a single question where students can select one answer from a pull-down list.

In [ ]:
# The information of the question
question_1 = "A large continental shelf width is at a:"
choices_1 = ["Leading edge", "Trailing edge", "Marginal sea"]
answer_1 = choices_1[1]
hint_1 = "Unfortunately not, here is a hint ..."
comment_1 = "Indeed, some additional information..."

# Make the required widgets
question_widget = pn.widgets.StaticText(value=question_1)
choices_widget = pn.widgets.Select(options=choices_1, name="")
submit_button = pn.widgets.Button(name="Submit")
feedback_widget = pn.widgets.TextInput(value="", name="", width=500)

# Align the widgets vertically
quiz_widget = pn.Column(question_widget, choices_widget, submit_button, feedback_widget)

def check_answers(event):
    chosen_answer = choices_widget.value
    correct_answer = answer_1

    if chosen_answer == correct_answer:
        feedback_widget.value = comment_1
    else:
        feedback_widget.value = hint_1

# Run the function check_answers when the submit button is pressed
submit_button.on_click(check_answers)

# display the panel
pn.panel(quiz_widget).servable()

This question can be improved by alligning the submit button and feedback widget next to each other, which can be done by pn.Row()

In [ ]:
def single_multiple_choice():
    # The information of the question
    question_1 = "A large continental shelf width is at a:"
    choices_1 = ["Leading edge", "Trailing edge", "Marginal sea"]
    answer_1 = choices_1[1]
    hint_1 = "Unfortunately not, here is a hint ..."
    comment_1 = "Indeed, some additional information..."

    # Make the required widgets
    question_widget = pn.widgets.StaticText(value=question_1)
    choices_widget = pn.widgets.Select(options=choices_1, name="")
    submit_button = pn.widgets.Button(name="Submit")
    feedback_widget = pn.widgets.TextInput(value="", name="", width=500)

    def check_answers(event):
        chosen_answer = choices_widget.value
        correct_answer = answer_1
    
        if chosen_answer == correct_answer:
            feedback_widget.value = comment_1
        else:
            feedback_widget.value = hint_1

    # Run the function check_answers when the submit button is pressed
    submit_button.on_click(check_answers)

    # Align the widgets
    submit_row = pn.Row(submit_button, feedback_widget)
    quiz_widget = pn.Column(question_widget, choices_widget, submit_row)
    display(quiz_widget)

single_multiple_choice()

#### Multiple questions

Ask multiple questions and give the total number of correct answers in the score.

In [ ]:
def Multiple_questions():
    # The information of the questions
    question_1 = "A large continental shelf width is at a:"
    choices_1 = ["Leading edge", "Trailing edge", "marginal sea"]
    answer_1 = choices_1[1]
    hint_1 = "Unfortunately not, here is a hint ..."
    comment_1 = "Indeed, .. some additional information ... "
    
    question_2 = "The oceanic geoid is: "
    choices_2 = [
        "An oval shape",
        "The shape of the ocean surface under only gravity forces",
        "A geo triangle with a different shape",
        "The actual ocean surface",
    ]
    answer_2 = choices_2[1]
    hint_2 = "Unfortunately not, here is a hint ..."
    comment_2 = "Indeed, .. some additional information ... "
    
    # store the questions in a list
    questions = [question_1, question_2]
    choices = [choices_1, choices_2]
    answers = [answer_1, answer_2]
    hints = [hint_1, hint_2]
    comments = [comment_1, comment_2]
    
    # An empty list to store the widgets
    all_widgets = []  # for visualization, store all the widgets in the order they are going to be displayed
    question_widgets = []  # store all the question widgets in a list
    choices_widgets = []  # store all the choices widgets in a list
    
    # make the widgets in a loop, one widget states the question and one shows the options that can be selected.
    for question, choice, answer, hint, comment in zip(questions, choices, answers, hints, comments):
        question_widget = pn.widgets.StaticText(value=question)
        choices_widget = pn.widgets.Select(options=choice, name="")
    
        question_widgets.append(question_widget)
        choices_widgets.append(choices_widget)
        all_widgets.append(question_widget)
        all_widgets.append(choices_widget)
    
    # make a submit button and a feedback button
    submit_button = pn.widgets.Button(name="Submit")
    feedback_widget = pn.widgets.TextInput(value="", name="")
    
    # allign the submit button and the feedback widget horizontally
    HBox_check = pn.Row(submit_button, feedback_widget)
    all_widgets.append(HBox_check)
    
    # unpacks the list by an asterix (*)
    quiz_widget = pn.Column(*all_widgets)
    
    # make a function to calculate the score and to give feedback
    def check_answers(event):
        score = 0
    
        for i in range(len(questions)):
            answer = choices_widgets[i].value
            correct_answer = answers[i]
        
            if answer == correct_answer:
                score += 1
        
        feedback_widget.value = "Your score is " + str(score) + "/" + str(len(questions))
    
    submit_button.on_click(check_answers)
    
    # display the panel
    display(quiz_widget)

Multiple_questions()

An improved version is below, giving the opportunity for feedback, randomizing the order of questions, and limiting the number of questions. If no feedback will be given

In [ ]:
def several_multiple_choice(questions, choices, answers, hints, comments, random_order = False, n_questions_max = 1e10):  
    # An empty list to store the widgets
    all_widgets = []  # For visualization, store all the widgets in the order they are going to be displayed
    question_widgets = []  # Store all the question widgets in a list
    choices_widgets = []  # store all the choices widgets in a list
    feedback_widgets = [] # store all the feedback widgets for subquestions

    # Optionally: randomize the order of the questions
    order = np.arange(0, len(questions), 1)
    if random_order == True:
        shuffle(order)

    # Optionally: limit the number of questions shown
    if len(questions) > n_questions_max:
        order = order[0:n_questions_max]
    
    # make the widgets in a loop, one widget states the question and one shows the options that can be selected.
    for i in np.array(order):
        question, choice, answer = questions[i], choices[i], answers[i]
        question_widget = pn.widgets.StaticText(value=question)
        choices_widget = pn.widgets.Select(options=choice, name="")
        feedback_widget =pn.widgets.StaticText(value="", name="")

        # store the widgets on type, make looping through them possible, the order is unaffected by randomizing
        question_widgets.append(question_widget)
        choices_widgets.append(choices_widget)
        feedback_widgets.append(feedback_widget)

        # store the widgets (again) for display purposes. It remains the same objects as in the previous list. 
        allign_question_feedback = pn.Row(choices_widget, feedback_widget)
        
        all_widgets.append(question_widget)
        all_widgets.append(allign_question_feedback)
    
    # make a submit button and a feedback button
    submit_button = pn.widgets.Button(name="Check")
    final_feedback_widget = pn.widgets.TextInput(value="", name="", width = 220)
    
    # Allign the submit button and the feedback widget horizontally
    HBox_check = pn.Row(submit_button, final_feedback_widget)
    all_widgets.append(HBox_check)
    
    # unpacks the list by an asterisk (*)
    quiz_widget = pn.Column(*all_widgets)
    
    # make a function to calculate the score and to give feedback
    def check_answers(event):
        score = 0
    
        for i in range(len(order)):
            answer = choices_widgets[i].value
            correct_answer = answers[i]
        
            if answer == correct_answer:
                score += 1
                feedback_widgets[i].value = comments[i]

            if answer != correct_answer:
                feedback_widgets[i].value = hints[i]
        
        final_feedback_widget.value = "Your score is " + str(score) + "/" + str(len(order))
    
    submit_button.on_click(check_answers)
    display(quiz_widget)

One example question is

In [ ]:
def ask_several_multiple_choice():
    # The information of the questions
    question_1 = "A large continental shelf width is at a:"
    choices_1 = ["Leading edge", "Trailing edge", "marginal sea"]
    answer_1 = choices_1[1]
    hint_1 = "Unfortunately not, here is a hint ..."
    comment_1 = "Indeed, .. some additional information ... "
    
    question_2 = "The oceanic geoid is: "
    choices_2 = [
        "An oval shape",
        "The shape of the ocean surface under only gravity forces",
        "A geo triangle with a different shape",
        "The actual ocean surface",
    ]
    answer_2 = choices_2[1]
    hint_2 = ""
    comment_2 = ""
    
    # store the questions in a list
    questions = [question_1, question_2]
    choices = [choices_1, choices_2]
    answers = [answer_1, answer_2]
    hints = [hint_1, hint_2]
    comments = [comment_1, comment_2]

    several_multiple_choice_feedback(questions, choices, answers, hints, comments, random_order = True)

ask_several_multiple_choice_feedback()

### Multiple selection

In the code below are correct and false statements presented. Students have to select the check boxes related to correct answers. Students gain points when they choose the correct answers but lose points if they check incorrect answers. The lowest score is 0 points.

An alternative check box that has a different layout. <br>
toggle = pn.widgets.Toggle(name='Toggle', button_type='success') <br>
https://panel.holoviz.org/reference/widgets/Toggle.html

In [ ]:
def multiple_selection():
    correct_statements = ["Correct", "The earth is round", "Good"]
    false_statements = ["False", "The earth is a cube"]
    
    # Make empty list to store the widgets (references), checkbox, and true/false statements sorted.
    check_boxes = []  # all the boxes to click
    all_statements = []  # all the statements
    
    # An empty list for visualization to store the HBoxes that contain the widgets, one statement and the corresponding checkbox
    all_widgets = []
    
    # Create Panel widgets
    pn.extension()
    
    for statement in correct_statements + false_statements:
        add_statement = pn.widgets.StaticText(value=statement, width=150)
        check_box_widget = pn.widgets.Checkbox(value=False, width=120)
        HBox1 = pn.Row(add_statement, check_box_widget)
    
        all_statements.append(add_statement)
        check_boxes.append(check_box_widget)
        all_widgets.append(HBox1)
    
    # randomize the order of statements
    shuffle(all_widgets)
    
    # add submit button and output, which come on the bottom
    submit_button = pn.widgets.Button(name='Submit')
    output_widget = pn.widgets.TextInput(value='', placeholder='', disabled=False)
    
    # make an additional HBox for aligning the submit button and the output widget
    HBox2 = pn.Row(submit_button, output_widget)
    all_widgets.append(HBox2)
    
    # align all the HBoxes beneath each other (oldest below if not randomized) and display them.
    quiz_widget = pn.Column(*all_widgets)
    
    # Check the checkbox for each statement and calculate the score.
    def check_answers(event):
        score = 0
    
        for i in range(len(check_boxes)):
            check_box = check_boxes[i]
            statement = all_statements[i].value
    
            if statement in correct_statements:
                if check_box.value == True:
                    score += 1
                else:
                    score -= 0
    
            if statement not in correct_statements:
                if check_box.value == True:
                    score -= 1
                else:
                    score -= 0
    
        score = np.max([score, 0])
        output_widget.value = 'Your final score is: ' + str(score)
    
    submit_button.on_click(check_answers)
    
    display(quiz_widget)

multiple_selection()

### Select correct statement

In [ ]:
def select_statement():

    # define the questions and store them in a list
    Q1 = 'Question 1'
    Q2 = 'Question 2'
    questions = [Q1, Q2]

    # the possible answers (may very per question)
    answers = ["Shallow", "Intermediate", "Deep"]

    # the correct answers, 0-based id
    correct_answers_id = [0,1]

    # define the widgets for visualization, make for each a row with question and answer
    # store all the rows with statements in a list for visualization, and in toggle_widgets for checking the answer
    Rows = []
    toggle_widgets = []
    for i in range(len(correct_answers_id)):
        question_widget = pn.widgets.StaticText(value=questions[i], width = 150)#statement
        
        radio_group_widget = pn.widgets.RadioButtonGroup(name='Radio Button Group', options=answers, button_type='default')
        toggle_widgets.append(radio_group_widget)
        
        add_row = pn.Row(question_widget, radio_group_widget)
        Rows.append(add_row)

    # randomize the order of statements
    shuffle(Rows)

    # add a submit button with a feedback option next to it
    submit_button =  pn.widgets.Button(name="Submit")
    feedback_widget = pn.widgets.StaticText(value="", name="", width=500)
    submit_row = pn.Row(submit_button, feedback_widget)
    
    # include a question
    text_general = "Select if the wave described on the right experiences shallow, intermediate, or deep water."
    text_widget = pn.widgets.StaticText(value=text_general)

    # structure the widgets and display thems
    display(text_widget, *Rows, submit_row)

    # check the answer and give feedback
    def check_answers(button):
        score = 0

        for i in range(len(correct_answers_id)):
            if toggle_widgets[i].value == answers[correct_answers_id[i]]:
                score += 1

        # print(toggle_widget.value)
        feedback_widget.value = "Your score is " + str(score) + "/" + str(len(correct_answers_id))

    submit_button.on_click(check_answers)

select_statement()

### Select figures

make a slider in which students can use a slider to select a figure. How path_figures can be defined using Git Hub have to be coordinated. The code is now as raw text to prevent it from running and giving an error notification that the figures are not found.

The figures and code that displays them through widgets was made by Mario van den Berg, for the test notebook about the escofier curves.

### Numerical questions

Two variants are made, one gives directly the correct answer andd one has the capability of giving a textual feedback.

#### Direct answer in feedback

In [ ]:
T1 = 6#s

Question = "Q1) What is the deep water wavelength?"
units = " m"
L = 9.81 * T1**2 / (2 * np.pi)
answer = round(L, 2)

question_widget = pn.widgets.StaticText(value=Question, width = 750)
unit_widget = pn.widgets.StaticText(value=units, width = 10)
num_widget = pn.widgets.FloatInput(value=0, step=0.01, width = 100)
feedback_widget = pn.widgets.TextInput(value="", name="", width=500)
submit_button =  pn.widgets.Button(name="Submit")

Hbox = pn.Row(num_widget, unit_widget, submit_button, feedback_widget)
quiz_widget = pn.Column(question_widget, Hbox)


def check_answers(button, answer = answer):
    # get value from widget and check if this corresponds with the answer
    response = num_widget.value

    if response == answer:  # if the answer is correct
        feedback_widget.value = str("Good job, this is correct")
    else:  # the answer is wrong
        feedback_widget.value = str("Incorrect, the answer should be " + str(answer) + str(units) + ", please try again.")

submit_button.on_click(check_answers)

display(quiz_widget)

#quiz_widget.servable() # display is recommended when it is put in a function.

#### Textual feedback

In [ ]:
T1 = 6#s

question = "Q1) What is the deep water wavelength?"
units = " m"
L = 9.81 * T1**2 / (2 * np.pi)
answer = round(L, 2)
Feedback_correct = 'Indeed, the deep water wavelength is in this way related to the wave period.'
Feedback_wrong = 'There is a mistake, the only variable is the wave period.'


question_widget = pn.widgets.StaticText(value=question, width = 750)
unit_widget = pn.widgets.StaticText(value=units, width = 10)
num_widget = pn.widgets.FloatInput(value=0, step=0.01, width = 100)
feedback_widget = pn.widgets.TextInput(value="", name="", width=500)
submit_button =  pn.widgets.Button(name="Submit")

Hbox = pn.Row(num_widget, unit_widget, submit_button, feedback_widget)
quiz_widget = pn.Column(question_widget, Hbox)


def check_answers(button, answer = answer):
    # get value from widget and check if this corresponds with the answer
    response = num_widget.value

    if response == answer:  # if the answer is correct
        feedback_widget.value = Feedback_correct
    else:  # the answer is wrong
        feedback_widget.value = Feedback_wrong

submit_button.on_click(check_answers)

display(quiz_widget)


### Coding question: Check values of parameter

Students can be asked to compute values of specific parameters in a code field. The students can then load their result into a widget, which has the option to check their results with the answers and gives feedback. It gives a notification if one of the parameters is not defined when loading. It also gives the correct answer after 3 attempts.

Below is the code that builds a User Interface (UI), assesses their input, and gives a response. It has to be defined/imported only once. It uses a class to store parameter values, as described in the section 'Working with widgets/parameters/variables in functions'.

In [ ]:
# general code, that has to be defined once
class class_variables:
    def __setattr__(self, key, value):
        object.__setattr__(self, key, value)

def classify_variables(params, params2 = {}, max_size_MB = 0):
    max_size = max_size_MB * 1024*1024
    FV = class_variables()
    for key, value in {**params,**params2}.items():
        if sys.getsizeof(value) < max_size or max_size <= 0:
            FV.__setattr__(key, value)
    return FV

def add_local_variables_to(FV,locals, max_size_MB = 0):
    max_size = max_size_MB * 1024*1024
    for key, value in  locals.items():
        if sys.getsizeof(value) < max_size or max_size <= 0:
            FV.__setattr__(key, value)
    return FV

def check_code_values(IV):
    # The function variable (FV) is now defined as Input Variable (IV) (= arguments)
    # The input variable and the newly defined local variable will be merged to FV

    
    def get_coded_values(FV,GV):
        def button_callback(b):
            try:
                for i, param in enumerate(FV.check_parameters):
    
                    # get the value that students gave
                    response = getattr(GV, param)
    
                    #store them in the widget
                    FV.all_parameter_widgets[i].value = response
                    
                FV.debug_widget.value = ''
            except:
                FV.debug_widget.value = '<b> Careful, not all parameters are defined! </b>'
    
        return button_callback

    def check_coded_values(FV,GV):
        def button_callback(b):
            FV.attempt.value += 1
            
            for i, param in enumerate(FV.check_parameters):
                #response = getattr(GV, param)
                response = FV.all_parameter_widgets[i].value
                answer = getattr(FV, param)
    
                if np.abs(response - answer) < FV.f_margin * answer:
                    FV.all_feedback_widgets[i].value = 'Nice, this is good!'
    
                if np.abs(response - answer) >= FV.f_margin * answer:
                    if FV.attempt.value < 3:
                        FV.all_feedback_widgets[i].value = 'This one is incorrect, try again!'
                    if FV.attempt.value >= 3:
                        FV.all_feedback_widgets[i].value = 'This one is incorrect, the answer should be ' + str(round(answer,FV.n_decimals)) + '.'
                
                #print('Debug: ', param, response, answer)
    
        return button_callback

    all_parameter_widgets = []
    info_widgets = []
    all_feedback_widgets = []
    for name, param in zip(IV.name_parameters, IV.check_parameters):
        symbol_widget = pn.widgets.StaticText(name='', value= name, width = 100)
        parameter_widget = pn.widgets.FloatInput(name='', value=0, width = 100)
        feedback_widget = pn.widgets.StaticText(value='')
        
        all_parameter_widgets.append(parameter_widget)
        all_feedback_widgets.append(feedback_widget)
        
        new_row = pn.Row(symbol_widget, parameter_widget, feedback_widget)
        info_widgets.append(new_row)

    debug_widget = pn.widgets.StaticText(value='')
    #attempt = pn.widgets.FloatInput(value=0)
                                 
    get_values_button =  pn.widgets.Button(name="Load values")
    check_values_button =  pn.widgets.Button(name="Check loaded values")

    GV = classify_variables(globals())
    FV = add_local_variables_to(IV,locals())

    # The error margin is set at 0.01% if it is not defined
    if 'f_margin' not in FV.__dict__:
        FV.__setattr__('f_margin', 0.0001)

    if 'n_decimals' not in FV.__dict__:
        FV.__setattr__('n_decimals', 10)
    
    get_values_button.on_click(get_coded_values(FV, GV))
    check_values_button.on_click(check_coded_values(FV, GV))
    
    row_buttons = pn.Row(get_values_button, check_values_button, debug_widget)
    Input_col = pn.Column(row_buttons, *info_widgets)
    display(Input_col)

The function below is used to ask the question to the students, it defines the given question and parameters, displays the question on the screen, and has a widget to store the number of attempts that students have made to check their answers.

The structure is:
- Give the question-rated parameters
- Define the question
- Make additional required widgets <b> (don't change them) </b>
- Make a globally defined parameter, you should give it a unique name.
- Contribute the question-related parameter and the attempt (widget) to the global variable.

The question-related parameters and the attempt counter are stored in a global variable, so the return and related print are prevented. The attempt (counter) is updated, which is simplified by making it a widget/object rather than a parameter.

In [ ]:
def Question():
    # the question-related parameters
    L1 = 6
    L2 = round(uniform(7,10), 1)

    # The question that is asked
    question = 'What is k1 and k2 when the lengts of wave 1 and 2 are ' + str(L1) + ' and ' + str(L2) + ' meter?'


    # make the attempt counter, one for each subquestion
    attempt = pn.widgets.FloatInput(value=0)
    # In case of multiple sub-question, this can be like.
    #attempt_A = pn.widgets.FloatInput(value=0)
    #attempt_B = pn.widgets.FloatInput(value=0)

    
    # Required widgets for functionality, should not be changed
    question_widget = pn.widgets.StaticText(name='', value= question)
    display(question_widget)

    # define a new global variable, with a unique name
    global W2_Q9_param 
    # Store the question-related parameters and the attempt widgets.
    # This can be done in a 'list;  (as below) or a class, with classify_variables()
    # The benefit of this way is that the 'list' of parameters can be copied-paste
    # The benefit of classes is that it is less prone to errors when other functions/questions use them.
    W2_Q9_param = L1, L2, attempt # or with sub-questions: W2_Q9_param = L1, L2, attempt_A, attempt_B

The following function is used to check the answers the students give. <br>
It has the following structure: <br>
- Load the parameters stored in the global variable.
  - Make sure that the attempt is named correctly, in case multiple similar widgets are defined.
- Define the parameter names that have to be checked
- Define how these parameters are shown on the screen, think about spaces and underscores.
- Compute the answers
- A standard part that calls the basic function, which should not be changed.

In [ ]:
def Check_Question():
    # Load the variables, and make sure the attempt is related to the correct counter when there are sub-questions, like attempt = attempt_B
    L1, L2, attempt = W2_Q9_param
    
    # define the parameter names that have to be checked
    check_parameters = ['k1', 'k2']

    # define the names of the parameters as they are displayed
    name_parameters = ['k 1', 'k 2']

    # define the answers for the questions, which are described in check_parameters
    k1 = 2*np.pi/L1
    k2 = 2*np.pi/L2

    # additional settings
    n_decimals = 4 # decimals of the answer shown when 3 wrong answers
    f_margin = 0.1 # the allowed (error) margin compared to the answer, for example 0.0001 = 0.01%

    # a standard part that stores the parameters and displays the UI
    FV = classify_variables(locals())
    check_code_values(FV)

The part below comes in the notebooks/Jupyter Books: <br>
- It starts with displaying the question, through Question()
- A cell for the coding questions, it consist of the structure of the code and explanation. The parts students have to code are indicated by 3 dots or red text.

In [ ]:
Question()

In [ ]:
k1 = ...
k2 = 'fill this in'

Check_Question()

Below is the code when a student has filled in valid numbers, from which k1 is correct.

In [ ]:
L1 = 6
k1 = 2*np.pi/L1
k2 = 2

Check_Question()

### Coding question: Check and plot function

The function below loads the function the students make. It will plot this function if this is possible, otherwise it will give a warning. Further on are two generic functions that can be used to make questions. One is made to asses functions that return a single value, and one that returns a value as an array. The example of the second generic function does not work, unfortunately. The general code works in week 3.

In [ ]:
def Question():

    # define the parameters, which can be loaded from globals()
    a, b = 1,2

    # set the horizontal axis of the graph
    horizontal_axis = np.arange(0,10+1,1)

    # define the correct function and its values along the x-axis.
    def correct_function(a,b,x):
        y = a*x + b
        return y
        
    correct_answer = [correct_function(a,b,x) for x in horizontal_axis]

    # make the graph and show the correct answer
    fig = Figure((5,2.5))
    ax = fig.subplots()
    pane = pn.pane.Matplotlib(fig, dpi=100)
    
    #  Plot the answer if the students function is found
    try:
        GV = classify_variables(globals())
        student_function = GV.student_function
        student_answer = [student_function(a,b,x) for x in horizontal_axis]# the arguments should be defined
        line = ax.plot(horizontal_axis, student_answer, label = 'Your answer')
        line = ax.plot(horizontal_axis, correct_answer, label = 'Correct answer')
        ax.legend()
    except:
        text_failed = 'Almost there, \n your function can not be plotted, \n please try to fix the bug'
        x_ticks = ax.get_xticks()
        y_ticks = ax.get_yticks()
        text = ax.text(np.average(x_ticks),np.average(y_ticks), text_failed, fontsize=16, color = 'r', ha='center', va='center')

    # update the graph
    display(pane)

This is a very basic way of setting up the question. And will not give a valid graph.

In [ ]:
def student_function(a,b,x):
    y = ...
    return y

a,b,x, = 1,2,3

Question()

Here is a valid function, with a small error.

In [ ]:
def student_function(a,b,x):
    y = a*x + b*2
    return y

Question()

In the cell below is the code more generalized. The list comprehension requires that the arguments are defined. This is solved by building a string of this function with arguments and then evaluating this. In this way can also additional parameters be introduced when the student is completely wrong. Further more is the zorder used to improve the graph.

#### Generic formula based on single point

In [ ]:
def check_code_function(fig, horizontal_axis, function_name, correct_function, par_x_axis, f_margin = 0.001):
    # The error margin (f_margin) is set at 0.1% if it is not defined

    # make the graph and show the correct answer
    ax = fig.subplots()
    pane = pn.pane.Matplotlib(fig, dpi=100)
    
    #  Plot the answer if the students function is found
    try:
        # Load the student-made function from globals
        function = globals()[function_name]

        # Use the student made function to calculate their answer
        # Add the arguments to the name of the function, for eval()
        # + Replace the argument on the x-axis so that it can be assessed with list comprehension.
        # https://docs.python.org/3/library/inspect.html
        # https://peps.python.org/pep-0362/    
        sig = signature(function)
        sig_function = str(sig).replace(par_x_axis, 'par_x_axis')
        function = function_name + str(sig_function)
        student_answer = [eval(function) for par_x_axis in horizontal_axis]

        # get the function for the correct answer
        sig = signature(correct_function)    
        sig_function = str(sig).replace(par_x_axis, 'par_x_axis')
        correct_function2 = str(correct_function.__name__) + sig_function

        #calculate the correct answer
        correct_answer = []
        for par_x_axis in horizontal_axis:
            correct_answer.append(eval(correct_function2))
        # This code should work, but it does not
        #correct_answer = [eval(correct_function2) for par_x_axis in horizontal_axis]

        # plot the answers
        line = ax.plot(student_answer, label = 'Your answer', zorder = 1)
        line2 = ax.plot(correct_answer, label = 'Correct answer', zorder = 1)

        # check if the answer is correct and plot it before/below lines
        changes = np.array(student_answer) - np.array(correct_answer)
        inaccuracy = np.abs(1-np.array(correct_answer)/np.array(student_answer))

        # Plot comment if the answer is correct
        y_loc = (np.nanmean(correct_answer) + np.nanmin(correct_answer))/2
        x_loc = np.mean(ax.get_xticks())
        if np.nanmax(changes) == 0:      
            text = ax.text(x_loc, y_loc, 'Perfect!', fontsize=12, color = '#1b5a00', ha='center', va='center', zorder = 0)

        if np.nanmax(changes) != 0 and np.nanmax(inaccuracy) < f_margin:
            text = ax.text(x_loc, y_loc, 'Good!', fontsize=12, color = '#1b5a00', ha='center', va='center', zorder = 0)

        # set title and legend
        title = function_name + str(sig)
        title = title.replace('_', ' ')
        ax.set_title(title)
        ax.legend()
           
    except:
        text_failed = 'Almost there, \n your function can not be plotted, \n please try to fix the bug'
        x_ticks = ax.get_xticks()
        y_ticks = ax.get_yticks()
        text = ax.text(np.average(x_ticks),np.average(y_ticks), text_failed, fontsize=16, color = 'r', ha='center', va='center')

    # update the graph
    display(pane)

The question related to one function, now with less repetive code compared to the first case.

In [ ]:
def Question():
    # define the name of the function that the students will make
    function_name = "student_function"

    # define the name of the parameter plotted on the horizontal axis
    parameter_x_axis = 'x'

    # set the horizontal axis of the graph
    horizontal_axis = np.arange(0,10+1,1)

    # define the correct function
    def correct_function(a,b,x):
        y = a*x + b
        return y

    # set the acceptable computational error (ratio)
    f_margin = 0.001 # 0.001 = 0.01%

    # set the size of the figure
    fig = Figure((5,2.5))

    # call the function that builds the backend.
    check_code_function(fig, horizontal_axis, function_name, correct_function, parameter_x_axis, f_margin)

An arbitrary wrong answer, the student introduced a new parameter. Note that the x-axis starts at 0, which gives the divide by zero warning with c, which is irrelevant.

In [ ]:
a,b,x, = 1,2,3
c = 4
def student_function(a,b,x,c):
    y = a*x + b*2 - c/x
    return y
    
Question()

In the current function, the parameter of the x-axis should be correctly named. It can not plot the graph if the name of the horizontal axis variable is different. So it is recommended to give this parameter, together with the name of the function, to the students. It can be useful to mention that the other relevant parameters should be added in the arguments, it is not possible to have ... or a text in here. The basic layout, without additional explanation is shown below. Replacing more_arguments with dots (...) or a comment will cause an error, which is not necessarily a problem.

In [ ]:
def name_function(x, more_arguments):
    y = ...
    return y

#### Generic formula based on 1D space

The function that students make can also be such that it calculates the values for an array directly. The general function, that has to be loaded once is:

In [ ]:
def check_code_function_RANGE(fig, function_name, correct_function, f_margin = 0.001):
    # The error margin (f_margin) is set at 0.1% if it is not defined

    # make the graph and show the correct answer
    ax = fig.subplots()
    pane = pn.pane.Matplotlib(fig, dpi=100)
    
    #  Plot the answer if the student function is found
    try:
        # Load the student-made function from globals
        function = globals()[function_name]

        # Add the arguments to the name of the function, for eval()
        # https://docs.python.org/3/library/inspect.html
        # https://peps.python.org/pep-0362/    
        sig = signature(function)
        student_function = function_name + str(sig)
        #print(student_function)

        student_answer = eval(student_function)# This line does not work, but does in week 3.
        #student_answer = x_range
        #print(student_answer)
        
        sig = signature(correct_function)    
        correct_function2 = str(correct_function.__name__) + str(sig)
        correct_answer = eval(correct_function2)

        # plot the answers
        line = ax.plot(student_answer, label = 'Your answer', zorder = 1)
        line = ax.plot(correct_answer, label = 'Correct answer', zorder = 1)

        # check if the answer is correct and plot it before/below lines
        changes = np.array(student_answer) - np.array(correct_answer)
        inaccuracy = np.abs(1-np.array(correct_answer)/np.array(student_answer))

        # Plot comment if the answer is correct
        y_loc = (np.nanmean(correct_answer) + np.nanmin(correct_answer))/2
        x_loc = np.mean(ax.get_xticks())
        if np.nanmax(changes) == 0:      
            text = ax.text(x_loc, y_loc, 'Perfect!', fontsize=12, color = '#1b5a00', ha='center', va='center', zorder = 0)

        if np.nanmax(changes) != 0 and np.nanmax(inaccuracy) < f_margin:
            text = ax.text(x_loc, y_loc, 'Good!', fontsize=12, color = '#1b5a00', ha='center', va='center', zorder = 0)

        # show legend, set title, and update figure
        ax.legend()
        title = student_function
        title = title.replace('_', ' ')
        ax.set_title(title)

    except:
        text_failed = 'Almost there, \n your function can not be plotted, \n please try to fix the bug'
        x_ticks = ax.get_xticks()
        y_ticks = ax.get_yticks()
        text = ax.text(np.average(x_ticks),np.average(y_ticks), text_failed, fontsize=16, color = 'r', ha='center', va='center')

    # update the graph
    display(pane)

Here is an example of an code, unfortunalely an error exist, the code stops when it evaluates the string describing the function. The code above works in week 3.

In [ ]:
def Question2():
    # define the name of the function that the students will make
    function_name = 'student_function'

    # define the correct function
    def correct_function(x_range,a,b):
        x = x_range
        y = a*x + b
        return y
    
    # set the acceptable computational error (ratio)
    f_margin = 0.001 # 0.001 = 0.01%

    # set the size of the figure
    fig = Figure((5,2.5))

    # call the function that builds the backend.
    check_code_function_RANGE(fig, function_name, correct_function, f_margin)


In [ ]:
x_range = np.arange(0.1,10+1,1)
a = 1
b = 3
c = 5

def student_function(x_range, a,b,c):
    y = a*x_range + b*2 - c/x_range
    return y

# a plot, showing that the student function works.
#plt.plot(x_range, student_function(x_range, a,b,c))

Question2()

In [ ]:
'''the code stops when it evaluates the string describing the function, however, it works below. '''

ans = eval('student_function(x_range, a, b ,c)')
print(ans)

#### Using subplots and suggested new structure by returning ax and pane 

Below is the latest version in making subplots, including the student's answer. 

It is suggested to simplify this function, and remove the inputs and plots for xlabel, ylabel, if it is a new graph etc. So, the function will only check and plot the answer at a predifend location. All the other things (like xlabel) can be defined in the function that builds the whole question, in a similar way the legend is plotted below.

In [ ]:
def check_code_function_RANGE(fig, function_name, correct_function, f_margin = 0.001, xlabel = None, ylabel = None, index_answer = '', new_graph = True, ax = None, pane = None, show_legend = True):
    # The error margin (f_margin) is set at 0.1% if it is not defined

    if new_graph == True:
        pane = pn.pane.Matplotlib(fig, dpi=100)
        ax = fig.subplots()

    if xlabel != None:
        ax.set_xlabel(xlabel)
        fig.subplots_adjust(bottom=0.25)

    if ylabel != None:
        ax.set_ylabel(ylabel)
        fig.subplots_adjust(left=0.15)  
    
    #  Plot the answer if the student function is found
    try:
        # Load the student-made function from globals
        function = globals()[function_name]

        # Add the arguments to the name of the function, for eval()
        # https://docs.python.org/3/library/inspect.html
        # https://peps.python.org/pep-0362/    
        sig = signature(function)
        student_function = function_name + str(sig)
        student_answer = eval(student_function)        
        #student_answer = eval(student_function)[0]

        
        sig = signature(correct_function)    
        correct_function2 = str(correct_function.__name__) + str(sig)
        #correct_answer = eval(correct_function2)
        
        if index_answer == '':
            student_answer = eval(student_function)
            correct_answer = eval(correct_function2)
        else:
            student_answer = eval(student_function)[index_answer]
            correct_answer = eval(correct_function2)[index_answer]

        # plot the answers
        line = ax.plot(student_answer, label = 'Your answer', zorder = 1)
        line = ax.plot(correct_answer, label = 'Correct answer', zorder = 1)

        # check if the answer is correct and plot it before/below lines
        changes = np.array(student_answer) - np.array(correct_answer)
        student_answer[student_answer == 0] = 1e-100 # avoid dividing by zero
        inaccuracy = np.abs(1-np.array(correct_answer)/np.array(student_answer))

        # Plot comment if the answer is correct
        y_loc = (np.nanmean(correct_answer) + np.nanmin(correct_answer))/2
        x_loc = np.mean(ax.get_xticks())
        if np.nanmax(changes) == 0:      
            text = ax.text(x_loc, y_loc, 'Perfect!', fontsize=12, color = '#1b5a00', ha='center', va='center', zorder = 0)

        if np.nanmax(changes) != 0 and np.nanmax(inaccuracy) < f_margin:
            text = ax.text(x_loc, y_loc, 'Good!', fontsize=12, color = '#1b5a00', ha='center', va='center', zorder = 0)

        # show legend, 
        if show_legend == True:
            ax.legend()

        # set title
        title = student_function
        title = title.replace('_', ' ')
        ax.set_title(title)

    except:
        text_failed = 'Almost there, \n your function can not be plotted, \n please try to fix the bug'
        x_ticks = ax.get_xticks()
        y_ticks = ax.get_yticks()
        text = ax.text(np.average(x_ticks),np.average(y_ticks), text_failed, fontsize=16, color = 'r', ha='center', va='center')

    # update the graph
    if new_graph == True:
        display(pane)
    else:
        return pane, ax

An example is below

In [ ]:
def W0_plot_function():
    
    # define the name of the function that the students will make
    function_name = "W0_function"

    # define the correct function
    def correct_function(x_range, h0, slope):
            x = x_range               # the horizontal axis
            zbed = -(h0 - slope * x)  # bed elevation [m]
            h = -zbed                 # still water depth [m]
            h[h < 0] = 0              # no negative depths
            
            return x, zbed, h
    
    # set the acceptable computational error (ratio)
    f_margin = 0.01 # 0.01 = 0.1%

    # set the size of the figure
    fig, axs = plt.subplots(nrows=3,ncols=1,figsize=(6,4), sharex=True, sharey = False)
    pane = pn.pane.Matplotlib(fig, dpi=100)

    # call the function that builds the backend.
    new_graph = False
    ax = axs[0]
    pane, ax = check_code_function_RANGE(fig, function_name, correct_function, f_margin = 0.01, xlabel = None, ylabel = 'x [m]', index_answer = 0, new_graph = new_graph, ax = ax, pane = pane, show_legend = False)
    ax.legend(bbox_to_anchor=(-0.02,1), loc="lower left")
    ax.set_title('')
    
    ax = axs[1]
    pane, ax = check_code_function_RANGE(fig, function_name, correct_function, f_margin = 0.01, xlabel = None, ylabel = 'zbed [m]', index_answer = 1, new_graph = new_graph, ax = ax, pane = pane, show_legend = False)
    ax.set_title('')

    ax = axs[2]
    pane, ax = check_code_function_RANGE(fig, function_name, correct_function, f_margin = 0.01, xlabel = None, ylabel = 'h [m]', index_answer = 2, new_graph = new_graph, ax = ax, pane = pane, show_legend = False)
    ax.set_title('')

And the application, including student's answer:

In [ ]:
x_range = np.arange(0,2000+1,1)
h0 = 10
slope = 1/50

def W0_function(x_range, h0, slope):
        x = x_range               # the horizontal axis
        zbed = -h0 + slope * x    # bed elevation [m]
        h = -zbed                 # still water depth [m]
        h[h < 0] = 0              # no negative depths

        h += 1 # an error, to deviate from the answer
    
        return x, zbed, h

W0_plot_function()

### Moving graph

Three different options for making moving graphs are depicted below. The recommended way is using Discrete Player.

The FuncAnimation works to make graphs, the challenge is to stop the graph (for which automatically a button came in %notebook). So for this is a self-made button required.
ipw.Play() may be a good alternative. However, the challenge is here if you want to have steps smaller than 1.

#### FuncAnimation

The funcanimation is an interesting tool for displaying graphs that move over time. One downside is that a button has to be coded that can be used to stop the graph. Otherwise it will be running the entire time while it is quite computationally demanding.

In [ ]:
%matplotlib widget
# manual set interval between frames in seconds
delta_t = 0.1

# setup linear mesh (x) and duration before time (t) is reset
x = np.linspace(0, 100, 1000)#  m
t = np.arange(0,1000,delta_t) #

a = 1
w = 0.35
k = 0.25

# Create figure and axes
fig, ax = plt.subplots()

# Compute initial displacement
eta =  a*np.sin(k*x)

# Plot initial wave
line, = ax.plot(x, eta)

# update the line for each frame
def update(frame):
    t = delta_t * frame
    eta = a*np.sin(w*t-k*x)
    line.set_ydata(eta)

    # usefull in testing, stop the graph at frame 100 without the need for a button/widget
    #if frame > 50:
    #    animation.event_source.stop()

# Create animation
animation = FuncAnimation(fig, update, frames=len(t), interval=delta_t*1000)

# create stop button that stops the graph
stop_button = ipw.Button(description="stop")
display(stop_button)

def stop_graph(button):
    animation.event_source.stop()
    plt.close() #  deletes the graph that has been stopped, to prevent making multiple graphs that fill the memory)
    
stop_button.on_click(stop_graph)

#### play widget

The play widget tool has inbuild options to play and stop the widget. However the stepsize is 1, even when the related widget has a finer resolution. So this tool is not suitable when it should give a smooth graph. The slider below is used for time, which means that the finest temporal resolution is one second.

In [ ]:
#%matplotlib widget
# set input parameters (can be made with widgets)
a = 1
w = 0.35
k = 0.25

# Set inital conditions
t = 0
x = np.linspace(0, 100, 1000)  # m
eta = a * np.sin(w*t - k*x)

# Create figure, axes, and initial values
fig, ax = plt.subplots()
line, = ax.plot(x, eta)

play = ipw.Play(
    value=0,
    min=0,
    max=100,
    step=1,
    interval=500, #  miliseconds
    description="Press play",
    disabled=False,
)

slider = ipw.FloatSlider(min=0, max=100, step=0.1)
ipw.jslink((play, 'value'), (slider, 'value'))


def update_line(change):
    t = change.new
    eta = a * np.sin(w * t - k * x)
    line.set_ydata(eta)
    fig.canvas.draw()


slider.observe(update_line, names='value')

ipw.HBox([play, slider])

#### Discrete player

The discrete player is the third option that has been studied. It gives the option to have a fine temporal resolution and an inbuild tool to stop the graph. Not all the options have been studied.

In [ ]:
# set input parameters (can be made with widgets)
a = 1
w = 0.35
k = 0.25
delta_t = 30 # ms

# Set initial conditions
t = 0
x = np.linspace(0, 100, 1000)  # m
eta = a * np.sin(w * t - k * x)

# Create figure, axes, and initial values
fig, ax = plt.subplots()
line, = ax.plot(x, eta)

def update_line(change):
    t = change.new
    eta = a * np.sin(w * t - k * x)
    line.set_ydata(eta)
    fig.canvas.draw()
    #print(t)

discrete_player = pn.widgets.DiscretePlayer(name='Discrete Player', options=np.arange(0,500,delta_t/1000).tolist(), value=0, loop_policy='loop', interval = delta_t)
discrete_player.param.watch(update_line, 'value')

pn.panel(discrete_player)

### Interactive graph and spline

A widget that can be used to set points on a graph with a left mouse click. And remove the closest point to the cursur by a right mouse click. 

A Spline will be fit through the dots. 

In [ ]:
#import numpy as np
#import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

https://pythonnumericalmethods.berkeley.edu/notebooks/chapter17.03-Cubic-Spline-Interpolation.html

#### Only dots with spline

In [ ]:
%matplotlib widget

# list to store x and y coordinates and the lines
points_x = [0,10]
points_y = [1,1]

# list of drawn objects
lines = []

def mouse_click(event):

    
    if event.button == 1:  # 1 Left mouse button, 3 right mouse button
        points_x.append(event.xdata)
        points_y.append(event.ydata)
        
        scatter1 = ax.scatter(points_x, points_y, c='k', s=40)
        scatter2 = ax.scatter(points_x, points_y, c='w', s=20)
              
        if len(points_x) > 1:
            
            # group the points
            coords = zip(points_x, points_y)

            # sort the points on x-coordinate
            sorted_pairs = sorted(coords, key=lambda parameter: parameter[0])
            sorted_x, sorted_y = zip(*sorted_pairs)
            
            # define cubic spline interpolation
            f = CubicSpline(sorted_x, sorted_y, bc_type='natural')
            x_new = np.linspace(0, 10, 1000)
            y_new = f(x_new)
                       
            # remove previous lines
            for line in lines:
                line.remove()
            lines.clear()
                        
            # plot the new line
            line, = ax.plot(x_new,y_new, linestyle='-', color = 'k')
            lines.append(line)
            
            
    if event.button == 3:  # 1 Left mouse button, 3 right mouse button
        
        # get mouse location
        mouse_x = event.xdata
        mouse_y = event.ydata

        # get the distances from all the points to the mouse in a list
        distance = []
        for x,y in zip(points_x, points_y):
            distance.append( ((x-mouse_x)**2 + (y-mouse_y)**2)**0.5)

        # get the id of the point nearest to the mouse
        id_low = distance.index(min(distance))
        
        if id_low != 0:# and id_low != (len(points_x)):

            # delete the closest point
            del(points_x[id_low])
            del(points_y[id_low])
        
            # regroup the points
            coords = zip(points_x, points_y)

            # sort the points on x-coordinate
            sorted_pairs = sorted(coords, key=lambda parameter: parameter[0])
            sorted_x, sorted_y = zip(*sorted_pairs)

            # define new cubic spline interpolation
            f = CubicSpline(sorted_x, sorted_y, bc_type='natural')
            x_new = np.linspace(0, 10, 100)
            y_new = f(x_new)

            # Clear the plot and reset axis settings
            ax.clear()  
            ax.set_xlim(0, 10)
            ax.set_ylim(0, 3)
            ax.set_xlabel('Alongshore location (x) [m]')
            ax.set_ylabel('sediment transport rate magnitude')

            ## remove previous lines
            for line in lines:
                line.remove()
            lines.clear()

            # plot the remaining dots and new spline
            scatter1 = ax.scatter(points_x, points_y, c='k', s=40)
            scatter2 = ax.scatter(points_x, points_y, c='w', s=20)

            line, = ax.plot(x_new,y_new, linestyle='-', color = 'k')
            lines.append(line)

            fig.canvas.draw()

    # debug
    #ax.text(0.02, 0.95, y_new, transform=ax.transAxes, fontsize=5, bbox={'facecolor': 'white'})

# Create figure
fig, ax = plt.subplots()
fig.canvas.toolbar_visible = False # dont show toolbar
ax.set_xlim(0, 10)
#ax.set_ylim(0, 3)

# make initial plot
scatter1 = ax.scatter(points_x, points_y, c='k', s=40)
scatter2 = ax.scatter(points_x, points_y, c='w', s=20)

line, = ax.plot(points_x,points_y, linestyle='-', color = 'k')
lines.append(line)

# activate the function mouse_click when a button is pressed
fig.canvas.mpl_connect('button_press_event', mouse_click);

plt.xlabel('Alongshore location (x) [m]')
plt.ylabel('sediment transport rate magnitude')

plt.show();

#### Dots with spline and gradient plotted

In [ ]:
%matplotlib widget

# list to store x and y coordinates and the lines
points_x = [0,10]
points_y = [1,1]

# list of drawn objects
lines = []
lines_gradient = []

def calc_gradient(x,y):
    dx = np.diff(x)
    dy = np.diff(y)
    print(len(x), len(dx))
    return dy/dx


def mouse_click(event):
    ax = axs[0]
    
    if event.button == 1 and ax.contains(event)[0]:  # 1 Left mouse button, within figure boundary
        points_x.append(event.xdata)
        points_y.append(event.ydata)
        
        scatter1 = ax.scatter(points_x, points_y, c='k', s=40)
        scatter2 = ax.scatter(points_x, points_y, c='w', s=20)
              
        if len(points_x) > 1:
            
            # group the points
            coords = zip(points_x, points_y)

            # sort the points on x-coordinate
            sorted_pairs = sorted(coords, key=lambda parameter: parameter[0])
            sorted_x, sorted_y = zip(*sorted_pairs)
            
            # define cubic spline interpolation
            f = CubicSpline(sorted_x, sorted_y, bc_type='natural')
            x_new = np.linspace(0, 10, 1000)
            y_new = f(x_new)
                       
            # remove previous lines
            for line in lines:
                line.remove()
            lines.clear()
                        
            # plot the new line
            line, = ax.plot(x_new,y_new, linestyle='-', color = 'k')
            lines.append(line)

            # plot gradient            

            for line in lines_gradient:
                line.remove()
            lines_gradient.clear()

            y_gradient = np.gradient(y_new,x_new)
            gradient, = axs[1].plot(x_new,y_gradient, linestyle='-', color = 'k')
            lines_gradient.append(gradient)
            
            
    if event.button == 3 and ax.contains(event)[0] and len(points_x) >= 3: #3 right mouse button, within figure boundary, and at least 3 points
        
        # get mouse location
        mouse_x = event.xdata
        mouse_y = event.ydata

        # get the distances from all the points to the mouse in a list
        distance = []
        for x,y in zip(points_x, points_y):
            distance.append( ((x-mouse_x)**2 + (y-mouse_y)**2)**0.5)

        # get the id of the point nearest to the mouse
        id_low = distance.index(min(distance))
        
        if id_low != 0:

            # delete the closest point
            del(points_x[id_low])
            del(points_y[id_low])
        
            # regroup the points
            coords = zip(points_x, points_y)

            # sort the points on x-coordinate
            sorted_pairs = sorted(coords, key=lambda parameter: parameter[0])
            sorted_x, sorted_y = zip(*sorted_pairs)

            # define new cubic spline interpolation
            f = CubicSpline(sorted_x, sorted_y, bc_type='natural')
            x_new = np.linspace(0, 10, 100)
            y_new = f(x_new)

            # Clear the plot and reset axis settings
            ax.clear()  
            ax.set_xlim(0, 10)
            ax.set_ylim(0, 3)
            ax.set_xlabel('Alongshore location (x) [m]')
            ax.set_ylabel('sediment transport rate magnitude')

            ## remove previous lines
            for line in lines:
                line.remove()
            lines.clear()

            # plot the remaining dots and new spline
            scatter1 = ax.scatter(points_x, points_y, c='k', s=40)
            scatter2 = ax.scatter(points_x, points_y, c='w', s=20)

            line, = ax.plot(x_new,y_new, linestyle='-', color = 'k')
            lines.append(line)

            fig.canvas.draw()

    # debug
    #axs[0].text(0.02, 0.95, y_new, transform=ax.transAxes, fontsize=5, bbox={'facecolor': 'white'})

# Create figure
fig, axs = plt.subplots(2,1, sharex = True)
fig.canvas.toolbar_visible = False
axs[0].set_xlim(0, 10)


# make initial plot
scatter1 = axs[0].scatter(points_x, points_y, c='k', s=40)
scatter2 = axs[0].scatter(points_x, points_y, c='w', s=20)

axs[1].axhline(0, color='silver', linestyle='--')
y_gradient = np.gradient(points_y,points_x)
gradient, = axs[1].plot(points_x,y_gradient, linestyle='-', color = 'k')
lines_gradient.append(gradient)

line, = axs[0].plot(points_x,points_y, linestyle='-', color = 'k')
lines.append(line)

# activate the function mouse_click when a button is pressed
fig.canvas.mpl_connect('button_press_event', mouse_click);

axs[1].set_xlabel('Alongshore location (x) [m]')
axs[0].set_ylabel('sediment transport \n rate magnitude')
axs[1].set_ylabel('sediment transport \n rate gradient')

plt.show();

### Adding changing plots as widgets

Plots can be redrawn when parameters are changed. This can be done with the [Param component](https://panel.holoviz.org/reference/panes/Param.html) of panel. The code below shows the implementation of having a graph updated when a (submit) button is pressed. The graphs are added similarly to other widgets, now with additional attention to defining the axis of the graph (ax), in which the figure is displayed (fig), and the panes widget (pane). The code below shows them for making two graphs. 

It is not recommended to have calculations etc. It is hard to return values. The simplest way (or workaround) is by using widgets and changing the value. Below is the counter used as input to change the value of the graph each time the button is pressed. Further on in this notebook is a section related to working with parameters in functions.

In [ ]:
from matplotlib.figure import Figure

In [ ]:
#set plot settings and make plots   
titles = ['plot1', 'graph2']
figures = []
plots = []
panes = []
for title in titles:
    fig = Figure((5,2.5))
    ax = fig.subplots()
    pane = pn.pane.Matplotlib(fig, dpi=96)

     # Add some extra space for labels at the axis
    fig.subplots_adjust(bottom=0.25) 
    fig.subplots_adjust(left=0.2)

    # store everything in a list
    figures.append(fig)
    plots.append(ax)
    panes.append(pane)

# set a function that defines what has to be done when the button is pressed
def plot_graph(figures, plots, panes, start_value_right, counter):

    def button_callback(b):
        ax = plots[0]
        ax.clear() # remove previous lines
        ax.plot(np.linspace(0,10,10), np.linspace(0,start_value_right + counter.value ,10), label = 'L [m]')
        ax.set_xlabel("x-label")
        panes[0].object = figures[0]

        ax = plots[1]
        ax.clear() # remove previous lines
        ax.plot(np.linspace(0,10,10), np.linspace(0,start_value_right - counter.value ,10), label = 'L [m]')
        ax.set_xlabel("x-label")
        panes[1].object = figures[1]

        counter.value += 1

    return button_callback  # otherwise gives TypeError: 'NoneType' object is not callable

start_value_right = 2

counter = pn.widgets.FloatInput(value=0)

submit_button =  pn.widgets.Button(name="Update graph")
submit_button.on_click(plot_graph(figures, plots, panes, start_value_right , counter))

pn.Column(*panes, submit_button)

### Improving numerical questions

The numerical questions are improved through various features/capabilities. The sections below improve the code stepwise, the final result will be used in the final cookbook.

#### Structure of widgets within code

In the code below are the widgets build within the function that also defines the questions. The same question is shown multiple times to visualize the impact of asking various questions.

##### <b> Display widgets seperatly <b/>

The function check_nummeric_answers can be left outside the function. The benefit of keeping it inside is that this function can be changed without that it has impact on other questions that call a function that is named similarly.

In [ ]:
def Q1():
    T1 = round(uniform(5, 8), 1)
    h1 = round(uniform(0.5, 5), 1)

    text_general = "Can you asses the wavelength in three different ways? Firstly through an iterative approach, then through tables (Appendix B, table B-3 of the book), and lastly via the formula of Fentom. The wave period ($T$) is " + str(T1) + " seconds, and the water depth ($h$) is " + str(h1) + " m?"
    text_widget = pn.widgets.StaticText(value=text_general, width = 750)
    display(text_widget)

    Q1_text = "Q1a) What is the deep water wavelength?"
    Q1_unit = " m"
    L = 9.81 * T1**2 / (2 * np.pi)
    Q1_answer = round(L, 2)
    Q1_FB_G = 'Indeed, the deep water wavelength is in this way related to the wave period.'
    Q1_FB_W = 'There is a mistake, the only variable is the wave period.'

    questions = [Q1_text, Q1_text,Q1_text]
    units = [Q1_unit, Q1_unit, Q1_unit]
    answers = [Q1_answer, Q1_answer, Q1_answer]
    FB_good = [Q1_FB_G, Q1_FB_G, Q1_FB_G]
    FB_wrong = [Q1_FB_W, Q1_FB_W, Q1_FB_W]
    
    # The code below does not have to be changed, only if the layout has to be changed.

    # A function that stores the values of the parameters when the submit button is made.
    def check_nummeric_answers(id, num_widget, answer, feedback_widget, FB_G, FB_W):

        def button_callback(b):
            #print("debug question:", id, ', response:' ,num_widget.value, ', answer:',answer)
    
            if answer == num_widget.value:
                feedback_widget.value = FB_G
            else:
                feedback_widget.value = FB_W
                print(num_widget.value)
    
        return button_callback  # required, otherwise, it gives TypeError: 'NoneType' object is not callable

    
    id = 0
    for question, units, answer, Q_FB_G, Q_FB_W in zip(questions, units, answers, FB_good, FB_wrong):
        id += 1
        question_widget = pn.widgets.StaticText(value=question, width = 750)
        unit_widget = pn.widgets.StaticText(value=units, width = 10)
        num_widget = pn.widgets.FloatInput(value=0, step=0.01, width = 100)
        feedback_widget = pn.widgets.TextInput(value="", name="", width=500)
        submit_button =  pn.widgets.Button(name="Submit")
        
        Hbox = pn.Row(num_widget, unit_widget, submit_button, feedback_widget)       
        quiz_widget = pn.Column(question_widget, Hbox)

        display(quiz_widget)

        # the varies 
        submit_button.on_click(check_nummeric_answers(id, num_widget, answer, feedback_widget, Q_FB_G, Q_FB_W))
Q1()

##### <b> Display widgets in one column  <b/>

The widgets are added into one column before displaying, which prevents that widgets can move relatively to one another.

In [ ]:
def check_nummeric_answers(id, num_widget, answer, feedback_widget, FB_G, FB_W):

    def button_callback(b):
        #print("debug question:", id, ', response:' ,num_widget.value, ', answer:',answer)

        if answer == num_widget.value:
            feedback_widget.value = FB_G
        else:
            feedback_widget.value = FB_W

    return button_callback  # otherwise gives TypeError: 'NoneType' object is not callable

def Q1():
    T1 = round(uniform(5, 8), 1)
    h1 = round(uniform(0.5, 5), 1)

    text_general = "Can you asses the wave length in three different ways? Firstly through an iterative approach, then through tables (Appendix B, table B-3 of the book), and lastly via the formula of Fentom. The wave period ($T$) is " + str(T1) + " seconds, and the water depth ($h$) is " + str(h1) + " m?"
    text_widget = pn.widgets.StaticText(value=text_general, width = 750)

    Q1_text = "Q1a) What is the deep water wavelength?"
    Q1_unit = " m"
    L = 9.81 * T1**2 / (2 * np.pi)
    Q1_answer = round(L, 2)
    Q1_FB_G = 'Indeed, the deep water wavelength is in this way related to the wave period.'
    Q1_FB_W = 'There is a mistake, the only variable is the wave period.'

    questions = [Q1_text, Q1_text,Q1_text]
    units = [Q1_unit, Q1_unit, Q1_unit]
    answers =[Q1_answer, Q1_answer, Q1_answer]
    FB_good = [Q1_FB_G, Q1_FB_G, Q1_FB_G]
    FB_wrong = [Q1_FB_W, Q1_FB_W, Q1_FB_W]
    
    all_widgets = []
    id = 0
    for question, units, answer, Q_FB_G, Q_FB_W in zip(questions, units, answers, FB_good, FB_wrong):
        id += 1
        question_widget = pn.widgets.StaticText(value=question, width = 750)
        unit_widget = pn.widgets.StaticText(value=units, width = 10)
        num_widget = pn.widgets.FloatInput(value=0, step=0.01, width = 100)
        feedback_widget = pn.widgets.TextInput(value="", name="", width=500)
        submit_button =  pn.widgets.Button(name="Submit")
        
        Hbox = pn.Row(num_widget, unit_widget, submit_button, feedback_widget)       
        quiz_widget = pn.Column(question_widget, Hbox)

        all_widgets.append(quiz_widget)
        

        # the values for the submit button are determined at the moment these are created.
        submit_button.on_click(check_nummeric_answers(id, num_widget, answer, feedback_widget, Q_FB_G, Q_FB_W))

    display(pn.Column(text_widget, *all_widgets))
Q1()

#### Call functions when building the question

Implementing functions can reduce repetition and thus the size of the code. The downside is that the code can become more complex, since the parameters have to be defined in the function and when calling it. Two different approaches are used below. First a function that contains the whole structure of setting up the questions and building the user interface. The second part consists of multiple functions that refer to each other. The benefit is that nesting is less required, which improves the structure and is therefore easier to read. However, it makes it harder to change since multiple questions can refer to the same function that you would like to change.

The widgets can be shown in various ways: 
- Display widgets individually in a loop, through display(...). It is easier to implement but could give the possibility that the widgets are not on a fixed position, relative to each other. 
- Make a list of widgets (in a loop) and display it in one command: <br>
    - through .servable(), it is more complex but allows previewing with Panel.
    - through display(), now with a small adjustment where an empty list is filled with the widgets in a loop, then combined in one column and displayed.

In [ ]:
def check_nummeric_answers(id, num_widget, answer, feedback_widget, FB_G, FB_W):

    def button_callback(b):
        #print("debug question:", id, ', response:' ,num_widget.value, ', answer:',answer)

        if answer == num_widget.value:
            feedback_widget.value = FB_G
        else:
            feedback_widget.value = FB_W

    return button_callback  # otherwise gives TypeError: 'NoneType' object is not callable


def nummeric_question_body(questions, units, answers, FB_good, FB_wrong):
    all_widgets = []
    id = 0
    for question, units, answer, Q_FB_G, Q_FB_W in zip(questions, units, answers, FB_good, FB_wrong):
        id += 1
        question_widget = pn.widgets.StaticText(value=question, width = 750)
        unit_widget = pn.widgets.StaticText(value=units, width = 10)
        num_widget = pn.widgets.FloatInput(value=0, step=0.01, width = 100)
        feedback_widget = pn.widgets.TextInput(value="", name="", width=500)
        submit_button =  pn.widgets.Button(name="Submit")
        
        Hbox = pn.Row(num_widget, unit_widget, submit_button, feedback_widget)       
        quiz_widget = pn.Column(question_widget, Hbox)

        all_widgets.append(quiz_widget)

        # the values for the submit button are determined at the moment these are created.
        submit_button.on_click(check_nummeric_answers(id, num_widget, answer, feedback_widget, Q_FB_G, Q_FB_W))
        
    return all_widgets


def Q1():
    T1 = round(uniform(5, 8), 1)
    h1 = round(uniform(0.5, 5), 1)

    text_general = "Can you asses the wave length in three different ways? Firstly through an iterative approach, then through tables (Appendix B, table B-3 of the book), and lastly via the formula of Fentom. The wave period ($T$) is " + str(T1) + " seconds, and the water depth ($h$) is " + str(h1) + " m?"
    text_widget = pn.widgets.StaticText(value=text_general, width = 750)
    
    Q1_text = "Q1a) What is the deep water wavelength?"
    Q1_unit = " m"
    L = 9.81 * T1**2 / (2 * np.pi)
    Q1_answer = round(L, 2)
    Q1_FB_G = 'Indeed, the deep water wavelength is in this way related to the wave period.'
    Q1_FB_W = 'There is a mistake, the only variable is the wave period.'

    questions = [Q1_text, Q1_text,Q1_text]
    units = [Q1_unit, Q1_unit, Q1_unit]
    answers =[Q1_answer, Q1_answer, Q1_answer]
    FB_good = [Q1_FB_G, Q1_FB_G, Q1_FB_G]
    FB_wrong = [Q1_FB_W, Q1_FB_W, Q1_FB_W]
    
    all_widgets = nummeric_question_body(questions, units, answers, FB_good, FB_wrong)
    
    display(pn.Column(text_widget,*all_widgets))
    
Q1()

#### Build a range for the answer

When students fillin more decimal numbers than asked lead to a response of an incorrect answer. Below is a function used to check the number of decimals in the given answer and uses this to check if the answer of students is within the boundaries.

In [ ]:
def limit_answer(x):
    s = str(x)

    # inspired by: https://stackoverflow.com/questions/35585950/find-the-number-of-digits-after-the-decimal-point
    if not '.' in s:
        n_decimal = 0
    else:
        n_decimal = len(s) - s.index('.') - 1

    range = 5*10**-(n_decimal+1)

    return range

In [ ]:
def check_nummeric_answers(id, num_widget, answer, feedback_widget, FB_G, FB_W):

    def button_callback(b):
        print("debug question:", id, ', response:' ,num_widget.value, ', answer:',answer)

        if np.abs(answer - num_widget.value) < limit_answer(answer):
            feedback_widget.value = FB_G
        else:
            feedback_widget.value = FB_W

    return button_callback  # otherwise gives TypeError: 'NoneType' object is not callable

def Q1():
    T1 = round(uniform(5, 8), 1)
    h1 = round(uniform(0.5, 5), 1)

    text_general = "Can you asses the wave length in three different ways? Firstly through an iterative approach, then through tables (Appendix B, table B-3 of the book), and lastly via the formula of Fentom. The wave period ($T$) is " + str(T1) + " seconds, and the water depth ($h$) is " + str(h1) + " m?"
    text_widget = pn.widgets.StaticText(value=text_general)#, width = 750)
    
    Q1_text = "Q1a) What is the deep water wavelength?"
    Q1_unit = " m"
    L = 9.81 * T1**2 / (2 * np.pi)
    Q1_answer = round(L, 2)
    Q1_FB_G = 'Indeed, the deep water wavelength is in this way related to the wave period.'
    Q1_FB_W = 'There is a mistake, the only variable is the wave period.'

    questions = [Q1_text, Q1_text,Q1_text]
    units = [Q1_unit, Q1_unit, Q1_unit]
    answers =[Q1_answer, Q1_answer, Q1_answer]
    FB_good = [Q1_FB_G, Q1_FB_G, Q1_FB_G]
    FB_wrong = [Q1_FB_W, Q1_FB_W, Q1_FB_W]
    
    all_widgets = nummeric_question_body(questions, units, answers, FB_good, FB_wrong)
    
    display(pn.Column(text_widget,*all_widgets))
    
Q1()

#### Give correct answer after x attempts

This section adds the possibility that after x attempts the correct answer is given. This question builds on the previous question where the answer should be inside a range (due to rounding)

In [ ]:
def check_nummeric_answers(id, answer, unit, FB_G, FB_W, num_widget, feedback_widget, attempt):
    
    def button_callback(b):
        attempt.value += 1
        print("debug question:", id, 'attempt', attempt.value ,', response:' ,num_widget.value, ', answer:',answer)

        # the answer is within the boundaries, print positive feedback
        if np.abs(answer - num_widget.value) < limit_answer(answer):
            if len(FB_G) != 0:
                feedback_widget.value = FB_G
            else:
                feedback_widget.value = 'Well done, this is correct!'

        # the answer is NOT within boundaries, provide feedback based on the number of attempts
        if np.abs(answer - num_widget.value) >= limit_answer(answer):

            if attempt.value < 3 and len(FB_W) > 0:
                feedback_widget.value = FB_W
                
            if attempt.value < 3 and len(FB_W) == 0:
                feedback_widget.value = 'Oops, there seems to be a mistake'
                
            if attempt.value >= 3:
                feedback_widget.value = 'The correct answer is ' + str(answer) + str(unit) + '.'


    return button_callback  # otherwise gives TypeError: 'NoneType' object is not callable

def nummeric_question_body(questions, units, answers, FB_good, FB_wrong):
    all_widgets = []
    attempts = []
    id = 0
    for question, units, answer, Q_FB_G, Q_FB_W in zip(questions, units, answers, FB_good, FB_wrong):
        id += 1
        question_widget = pn.widgets.StaticText(value=question, width = 750)
        unit_widget = pn.widgets.StaticText(value=units, width = 10)
        num_widget = pn.widgets.FloatInput(value=0, step=0.01, width = 100)
        feedback_widget = pn.widgets.TextInput(value="", name="", width=500)
        submit_button =  pn.widgets.Button(name="Submit")
        
        Hbox = pn.Row(num_widget, unit_widget, submit_button, feedback_widget)       
        quiz_widget = pn.Column(question_widget, Hbox)

        all_widgets.append(quiz_widget)

        # the values for the submit button are determined at the moment these are created.
        attempt = pn.widgets.FloatInput(value=0)
        attempts.append(attempt)
        submit_button.on_click(check_nummeric_answers(id, answer, units, Q_FB_G, Q_FB_W, num_widget, feedback_widget, attempt))
        
    return all_widgets

def Q1():
    T1 = round(uniform(5, 8), 1)
    h1 = round(uniform(0.5, 5), 1)

    text_general = "Can you asses the wave length in three different ways? Firstly through an iterative approach, then through tables (Appendix B, table B-3 of the book), and lastly via the formula of Fentom. The wave period ($T$) is " + str(T1) + " seconds, and the water depth ($h$) is " + str(h1) + " m?"
    text_widget = pn.widgets.StaticText(value=text_general, width = 750)
    
    Q1_text = "Q1a) What is the deep water wavelength?"
    Q1_unit = " m"
    L = 9.81 * T1**2 / (2 * np.pi)
    Q1_answer = round(L, 2)
    Q1_FB_G = 'Indeed, the deep water wavelength is in this way related to the wave period.'
    Q1_FB_W = 'There is a mistake, the only variable is the wave period.'

    questions = [Q1_text, Q1_text,Q1_text]
    units = [Q1_unit, Q1_unit, Q1_unit]
    answers =[Q1_answer, Q1_answer, Q1_answer]
    FB_good = [Q1_FB_G, Q1_FB_G, Q1_FB_G]
    FB_wrong = [Q1_FB_W, Q1_FB_W, Q1_FB_W]

    
    all_widgets = nummeric_question_body(questions, units, answers, FB_good, FB_wrong)
    
    display(pn.Column(text_widget,*all_widgets))
    
Q1()

#### Randomize order of questions

In [ ]:
def nummeric_question_body(questions, units, answers, FB_good, FB_wrong, random_order = False):
    all_widgets = []
    attempts = []

    order = np.arange(0, len(questions), 1)
    if random_order == True:
        shuffle(order)

    for i in np.array(order):
        question, unit, answer, Q_FB_G, Q_FB_W = questions[i], units[i], answers[i], FB_good[i], FB_wrong[i]
        id = i+1 
        question_widget = pn.widgets.StaticText(value=question, width = 750)
        unit_widget = pn.widgets.StaticText(value=unit, width = 10)
        num_widget = pn.widgets.FloatInput(value=0, step=0.01, width = 100)
        #feedback_widget = pn.widgets.TextInput(value="", name="", width=500)
        feedback_widget = pn.widgets.StaticText(value="", name="", width=500)
        submit_button =  pn.widgets.Button(name="Submit")
        
        Hbox = pn.Row(num_widget, unit_widget, submit_button, feedback_widget)       
        quiz_widget = pn.Column(question_widget, Hbox)

        all_widgets.append(quiz_widget)

        # the values for the submit button are determined at the moment these are created.
        attempt = pn.widgets.FloatInput(value=0)
        attempts.append(attempt)
        submit_button.on_click(check_nummeric_answers(id, answer, unit, Q_FB_G, Q_FB_W, num_widget, feedback_widget, attempt))
        
    return all_widgets


def Q1():
    T1 = round(uniform(5, 8), 1)
    h1 = round(uniform(0.5, 5), 1)

    text_general = "Can you asses the wave length in three different ways? Firstly through an iterative approach. The wave period ($T$) is " + str(T1) + " seconds, and the water depth ($h$) is " + str(h1) + " m?"
    text_widget = pn.widgets.StaticText(value=text_general, width = 750)
    
    Q1_text = "Q1a) What is the deep water wavelength?"
    Q1_unit = " m"
    L = 9.81 * T1**2 / (2 * np.pi)
    Q1_answer = round(L, 2)
    Q1_FB_G = 'Indeed, the deep water wavelength is in this way related to the wave period.'
    Q1_FB_W = 'There is a mistake, the only variable is the wave period.'

    questions = [Q1_text, Q1_text,Q1_text]
    units = [Q1_unit, Q1_unit, Q1_unit]
    answers =[Q1_answer, Q1_answer, Q1_answer]
    FB_good = [Q1_FB_G, Q1_FB_G, Q1_FB_G]
    FB_wrong = [Q1_FB_W, Q1_FB_W, Q1_FB_W]

    
    all_widgets = nummeric_question_body(questions, units, answers, FB_good, FB_wrong, random_order = True)
    
    display(pn.Column(text_widget,*all_widgets))
    
Q1()

#### Getting values of multiple nummerical widgets

The code below can be usefull and used to get the values of all the widgets that are stored in a list.
One case is the answers/responses provided by students in num_widgets, for example through:

In [ ]:
def return_answers_widgets(widgets):
    answers = []
    for widget in widgets:
        answers.append(widget.value)
    return answers

# And get the given answers to question 1.
#answers_question2 = return_answers_widgets(all_answers[1])

### Layout feedback widget

Various text widgets from panel can be used to give feedback, which have various benefits and downsizes

In [ ]:
feedback_widget1 = pn.widgets.TextInput(value="widget 1: same layout, height not flexible", name="", width=500)

feedback_widget2 = pn.widgets.TextAreaInput(value="widget 2, widget heigth flexibale, tekst different layout than at other widgets", name="", width=500, height = 60)
feedback_widget2.style = {'font-size': '4px'}# this does not work

feedback_widget3 = pn.widgets.StaticText(value='widget 3, height will change automatically to the content. The layout of the tekst is the same. No boundaries of the widget are shown.', width = 500)

display(pn.Column(feedback_widget1,feedback_widget2, feedback_widget3))

The height of the widget can be changed by adding a few lines at the bottom in the function check_nummeric_answers.

In [ ]:
def round_up(x, base=5):
    nr = base * (round(x//base)+1)
    nr = np.round(nr, 14)# to prevent values like .000000000000001
    return nr

def check_nummeric_answers(id, answer, unit, FB_G, FB_W, num_widget, feedback_widget, attempt):
    
    def button_callback(b):
        attempt.value += 1
        print("debug question:", id, 'attempt', attempt.value ,', response:' ,num_widget.value, ', answer:',answer)

        # the answer is within the boundaries, print positive feedback
        if np.abs(answer - num_widget.value) < limit_answer(answer):
            if len(FB_G) != 0:
                feedback_widget.value = FB_G
            else:
                feedback_widget.value = 'Well done, this is correct!'

        # the answer is NOT within boundaries, provide feedback based on the number of attempts
        if np.abs(answer - num_widget.value) >= limit_answer(answer):

            if attempt.value < 3:
                feedback_widget.value = FB_W
            else:
                feedback_widget.value = 'The correct answer is ' + str(answer) + str(unit) + '.'

        new_height = int(round_up(len(feedback_widget.value)/2, 40))
        feedback_widget.height = new_height
        print('debug: feedback widget height:', new_height)
        

    return button_callback  # otherwise gives TypeError: 'NoneType' object is not callable

### Working with widgets/parameters/variables in functions

It can become a challenge to manage all the parameters/widgets when functions are used. All the parameters (or variables in Python terminology) have to be defined in the function and described when the function is called. Especially when a list of widgets is used for various numerical questions that have various answers and graphs are plotted with the recent values. A workaround is to store the values of the local - or global-defined parameters into a single parameter that is given to a function. This parameter should then be unpacked or looked up in the function to make the calculation possible. Globals() are the parameters that are previous defined outside functions. Parameters that are defined inside functions are only local valid/stored, and are described with locals().

#### Parameters should be defined to change them

The code below shows that parameters should be defined between the brackets in order to work with them.

In [ ]:
global_variable = 1

def function():
    print(global_variable)

    # Executing the code below gives: UnboundLocalError: cannot access local variable 'global_variable' where it is not associated with a value
    # global_variable += 1

function()

The code below shows that it is possible to manage a few parameters by defining them as input parameter and defining them when calling the function. The below prints the input value when the function is called.

In [ ]:
global_variable = 1

def function(global_variable):
    print(global_variable)

    global_variable += 1
    return global_variable

global_variable = function(global_variable)
global_variable = function(global_variable)
global_variable = function(global_variable)

#### Storing all global and local parameters in one parameter

The function below stores all the global parameters in a single parameter, which is then given to the function.

In [ ]:
def function(all_parameters):
    global_variable = all_parameters['global_variable']
    print(global_variable)

    global_variable += 1
    return global_variable

all_parameters = globals()
global_variable = function(all_parameters)

Local variables are parameters are all the parameters that are valid within the (environment) of the function. The locals() consider only parameters that are defined in the function itself or are given to it (which are the values between the brackets)

In [ ]:
def function(all_parameters):
    global_variable = all_parameters['global_variable']
    global_variable += 1

    print(global_variable)

    new_local = 'local parameter'
    all_local_parameters = (locals())
    # the code below prints all the local parameters, with new_local on the bottom.
    #print(all_local_parameters)
    
    return global_variable

all_parameters = globals()
global_variable = function(all_parameters)

The code below stores all the local variables.

In [ ]:
def function():
    new_local = 'local parameter in the function'
    par1 = 5
    par2 = 4
    
    function_parameters = {key: value for key, value in locals().items()}
    # 

    print('The function_parameters')
    print(function_parameters)

    print('The value of par1:', function_parameters['par1'])

function()

Doing both the global and local parameters can be computationally demanding when it is inside a function in the following way (combining dictonaries):

In [ ]:
function_parameters = {key: value for key, value in {**globals(), **locals()}.items()}

In the following way can newly defined parameters be included.

In [ ]:
function_parameters = {key: value for key, value in {**globals(), **locals()}.items()}

def function():
    new_par = 4
    new_par2 = 5

    new_pars = {key: value for key, value in locals().items()}
    function_parameters.update(new_pars)

    print(function_parameters['new_par'])

function()

<b> Implementation in question structrure </b>

The code below shows that information can be passed on to nested functions, although parameters change. It also shows that it is not required to store all widgets in lists to have it functional.

In [ ]:
import panel as pn

def print_function(FV):
    def button_callback(event):
        print('button: ', FV['value'], 'value of widget', FV['num_widget'].value)

    return button_callback

def function():
    values = [1, 2, 3]

    def nested_function(values):
        rows = []
        for value in values:
            submit_button = pn.widgets.Button(name="Submit")
            num_widget = pn.widgets.FloatInput(value=0, step=0.01, width = 100)
            rows.append(pn.Row(submit_button, num_widget))

            function_parameters = {key: value for key, value in {**globals(), **locals()}.items()}
            submit_button.on_click(print_function(function_parameters))
               
        return rows

    widgets = nested_function(values)
    display(*widgets)

function()

This principle is applied to a numerical question in the code below. A debug print() function is added.

In [ ]:
def limit_answer(x):
    s = str(x)

    # inspired by: https://stackoverflow.com/questions/35585950/find-the-number-of-digits-after-the-decimal-point
    if not '.' in s:
        n_decimal = 0
    else:
        n_decimal = len(s) - s.index('.') - 1

    range = 5*10**-(n_decimal+1)

    return range

def check_nummeric_answers(FV, attempt):
    
    def button_callback(b):
        answer = FV['answer']
        unit = FV['unit']
        num_widget = FV['num_widget']
        feedback_widget = FV['feedback_widget']
        FB_W = FV['Q_FB_W']
        FB_G = FV['Q_FB_G']
       
        attempt.value += 1

        print('Debug: attempt:', attempt.value, ', Value of the widget:', num_widget.value)

        # the answer is within the boundaries, print positive feedback
        if np.abs(answer - num_widget.value) < limit_answer(answer):
            if len(FB_G) != 0:
                feedback_widget.value = FB_G
            else:
                feedback_widget.value = 'Well done, this is correct!'

        # the answer is NOT within boundaries, provide feedback based on the number of attempts
        if np.abs(answer - num_widget.value) >= limit_answer(answer):

            if attempt.value < 3 and len(FB_W) > 0:
                feedback_widget.value = FB_W
                
            if attempt.value < 3 and len(FB_W) == 0:
                feedback_widget.value = 'Oops, there seems to be a mistake'
                
            if attempt.value >= 3:
                feedback_widget.value = 'The correct answer is ' + str(answer) + str(unit) + '.'

    return button_callback  # otherwise gives TypeError: 'NoneType' object is not callable

def nummeric_question_body(FV, random_order = False):
    #FV = FV.copy()
    
    all_widgets = []
    attempts = []

    order = np.arange(0, len(FV['questions']), 1)
    if random_order == True:
        shuffle(order)

    for i in np.array(order):
        question, unit, answer, Q_FB_G, Q_FB_W = FV['questions'][i], FV['units'][i], FV['answers'][i], FV['FB_good'][i], FV['FB_wrong'][i]
        id = i+1 
        question_widget = pn.widgets.StaticText(value=question, width = 750)
        unit_widget = pn.widgets.StaticText(value=unit, width = 10)
        num_widget = pn.widgets.FloatInput(value=0, step=0.01, width = 100)
        #feedback_widget = pn.widgets.TextInput(value="", name="", width=500)
        feedback_widget = pn.widgets.StaticText(value="", name="", width=500)
        submit_button =  pn.widgets.Button(name="Submit")
        
        Hbox = pn.Row(num_widget, unit_widget, submit_button, feedback_widget)       
        quiz_widget = pn.Column(question_widget, Hbox)

        all_widgets.append(quiz_widget)

        # the values for the submit button are determined at the moment these are created.
        attempt = pn.widgets.FloatInput(value=0)
        attempts.append(attempt)

        FV2 = {key: value for key, value in {**globals(), **locals()}.items()}
        submit_button.on_click(check_nummeric_answers(FV2, attempt))
        
    return all_widgets


def Q1():
    T1 = round(uniform(5, 8), 1)
    h1 = round(uniform(0.5, 5), 1)

    text_general = "Can you asses the wave length in three different ways? Firstly through an iterative approach. The wave period ($T$) is " + str(T1) + " seconds, and the water depth ($h$) is " + str(h1) + " m?"
    text_widget = pn.widgets.StaticText(value=text_general, width = 750)
    
    Q1_text = "Q1a) What is the deep water wavelength?"
    Q1_unit = " m"
    L = 9.81 * T1**2 / (2 * np.pi)
    Q1_answer = round(L, 2)
    Q1_FB_G = 'Indeed, the deep water wavelength is in this way related to the wave period.'
    Q1_FB_W = 'There is a mistake, the only variable is the wave period.'

    questions = [Q1_text, Q1_text,Q1_text]
    units = [Q1_unit, Q1_unit, Q1_unit]
    answers =[Q1_answer, Q1_answer, Q1_answer]
    FB_good = [Q1_FB_G, Q1_FB_G, Q1_FB_G]
    FB_wrong = [Q1_FB_W, Q1_FB_W, Q1_FB_W]

    FV = {key: value for key, value in {**globals(), **locals()}.items()}
    all_widgets = nummeric_question_body(FV, random_order = True)
    
    display(pn.Column(text_widget,*all_widgets))
    
Q1()

#### Storing data in classes, for dot notation.

Shorter notation can help in having an overview of the code. The dot notification can also be useful, it will reduce the number of brackets etc. This can be achieved by working with classes. The code below gives an example. <br>

A shorter parameter name can also increase the readability, as long as the parameters are self-explanatory. A very short name can be used if it is constantly used for one specific goal, like storing all the local variables in it. Another way of naming parameters is by following naming conventions, such as the CamelCase. Here every new word in a parameter starts with a capital so spaces/underscores can be left out. In this project are some opinions that do not want capitals in naming, so that convention will not be used in this project.

The code below stores the parameter in globals() as a class. 

In [ ]:
class class_variables:
    def __setattr__(self, key, value):
        object.__setattr__(self, key, value)


In [ ]:
global_1 = 5
global_2 = 2

FV = class_variables()
for key, value in globals().items():
    FV.__setattr__(key, value)

print(FV.global_1, FV.global_2)

Now in a function, so it can be called.

In [ ]:
def classify_variables(params): 
    FV = class_variables()
    for key, value in params.items():
        FV.__setattr__(key, value)
    return FV

FV = classify_variables(globals())
print(FV.global_1, FV.global_2)

A similar thing can be done with locals(). The code below shows that both the local variables are not stored in globals() and vice versa. The name FV is used for Function_Variables, related to the default name in Python of locals within a function, which is the final goal of this section.

In [ ]:
def function():
    par1 = 5
    par2 = 4

    FV = classify_variables(globals())
    print(FV.global_1, FV.global_2)

    # gives no valid result
    #print(FV.par1)
    
    FV = classify_variables(locals())
    print(FV.par1)

    # gives no valid result
    #print(FV.global_1, FV.global_2)

function()

The function below is improved, so it can store locals and globals together.

In [ ]:
def classify_variables(params, params2 = None): 
    FV = class_variables()
    for key, value in params.items():
        FV.__setattr__(key, value)

    if len(params2) > 0:
        for key, value in params2.items():
            FV.__setattr__(key, value)
    return FV

def function():
    par1 = 5
    par2 = 4

    FV = classify_variables(globals(), locals())
    print(FV.global_1, FV.global_2)
    print(FV.par1)

function()

The global variables are always accessible, not necessarily editable. So this can be used to simplify the function.

In [ ]:
def classify_variables(locals): 
    FV = class_variables()
    for key, value in locals.items():
        FV.__setattr__(key, value)

    for key, value in globals().items():
        FV.__setattr__(key, value)
    return FV

def function():
    par1 = 5
    par2 = 4

    FV = classify_variables(locals())
    print(FV.global_1, FV.global_2)
    print(FV.par1)

function()

The code can be shortened by merging the dictonaries of globals() and locals().

In [ ]:
def classify_variables(locals): 
    FV = class_variables()
    for key, value in  {**globals(), **locals}.items():
        FV.__setattr__(key, value)

    return FV

def function():
    par1 = 5
    par2 = 4

    FV = classify_variables(locals())
    print(FV.global_1, FV.global_2)
    print(FV.par1)

function()

Storing the globals everytime might not be prefered, so that will be left out. It will be stored as parameters that will be unpacked.

In [ ]:
global_1 = 5
global_2 = 2

def classify_variables(params, params2 = {}): 
    FV = class_variables()
    for key, value in {**params,**params2}.items():
        FV.__setattr__(key, value)

    return FV

def function():
    par1_new = 5
    par2_new = 4

    FV = classify_variables(globals(), locals())
    print(FV.global_1, FV.global_2)
    print(FV.par1_new, FV.par2_new)

function()

Parameters can become memory demanding when data sets are stored in it, for example. It can be beneficial to only store files up to a specific size. This is applied in the code below. Here a maximum size is only considered when it is defined when calling the function. A max_size_MB smaller then or equal to 0 does not filter parameters based on its size.

In [ ]:
import sys

In [ ]:
def classify_variables(params, params2 = {}, max_size_MB = 0):
    max_size = max_size_MB * 1024*1024
    FV = class_variables()
    for key, value in {**params,**params2}.items():
        if sys.getsizeof(value) < max_size or max_size <= 0:
            FV.__setattr__(key, value)
    return FV

global_1 = 5
global_2 = 2

def function():
    par1_new = 5
    par2_new = 4

    FV = classify_variables(locals(), globals())
    print(FV.global_1, FV.global_2)
    print(FV.par1_new, FV.par2_new)

function()

Another aspect is that new parameters can be defined in nested functions. These have to be included to FV to pass on the information, unless the traditional way of passing parameters is used. This is probably the easiest way, since not many parameters will be defined in nested functions.

In [ ]:
def add_local_variables_to(FV,locals, max_size_MB = 0):
    max_size = max_size_MB * 1024*1024
    for key, value in  locals.items():
        if sys.getsizeof(value) < max_size or max_size <= 0:
            FV.__setattr__(key, value)
    return FV

def nested_function(FV):
    new_par1 = 9
    FV = add_local_variables_to(FV,locals())

    print(FV.new_par1)

def function():
    par1 = 5
    par2 = 4

    FV = classify_variables(locals())
    nested_function(FV)

function()

##### <b> implementing in numerical questions </b>

The code below shows the implementation in numerical questions. 

In [ ]:
import sys

In [ ]:
class class_variables:
    def __setattr__(self, key, value):
        object.__setattr__(self, key, value)

def classify_variables(params, params2 = {}, max_size_MB = 0):
    max_size = max_size_MB * 1024*1024
    FV = class_variables()
    for key, value in {**params,**params2}.items():
        if sys.getsizeof(value) < max_size or max_size <= 0:
            FV.__setattr__(key, value)
    return FV

# not required in this example
def add_local_variables_to(FV,params, max_size_MB = 0):
    max_size = max_size_MB * 1024*1024
    for key, value in  params.items():
        if sys.getsizeof(value) < max_size or max_size <= 0:
            FV.__setattr__(key, value)
    return FV

def limit_answer(answer, f_margin = 0):
    # if no margin is given, adding additional decimals are counted correctly.
    # if a margin if given, the answer should be within a range from the correct answer.
    # The maximal deviation is determined with the factor (f_margin)
    s = str(answer)

    if f_margin == 0: 
        # inspired by: https://stackoverflow.com/questions/35585950/find-the-number-of-digits-after-the-decimal-point
        if not '.' in s:
            n_decimal = 0
        else:
            n_decimal = len(s) - s.index('.') - 1
    
        range = 5*10**-(n_decimal+1)

    if f_margin != 0:
        range = f_margin * answer

    return range

def check_nummeric_answers(FV, attempt):
    
    def button_callback(b):
        answer = FV.answer
        unit = FV.unit
        num_widget = FV.num_widget
        feedback_widget = FV.feedback_widget
        FB_W = FV.Q_FB_W
        FB_G = FV.Q_FB_G
        
        attempt.value += 1
        #print('Debug: attempt:', attempt.value, ', Value of the widget:', num_widget.value)
        
        # the answer is within the boundaries, print positive feedback
        if np.abs(answer - num_widget.value) < limit_answer(answer):
            if len(FB_G) != 0:
                feedback_widget.value = FB_G
            else:
                feedback_widget.value = 'Well done, this is correct!'

        # the answer is NOT within boundaries, provide feedback based on the number of attempts
        if np.abs(answer - num_widget.value) >= limit_answer(answer):

            if attempt.value < 3 and len(FB_W) > 0:
                feedback_widget.value = FB_W
                
            if attempt.value < 3 and len(FB_W) == 0:
                feedback_widget.value = 'Oops, there seems to be a mistake'
                
            if attempt.value >= 3:
                feedback_widget.value = 'The correct answer is ' + str(answer) + str(unit) + '.'


    return button_callback  # otherwise gives TypeError: 'NoneType' object is not callable

def nummeric_question_body(FV, random_order = False):
    #FV = FV.copy()
    
    all_widgets = []
    attempts = []

    order = np.arange(0, len(FV.questions), 1)
    if random_order == True:
        shuffle(order)

    for i in np.array(order):
        question, unit, answer, Q_FB_G, Q_FB_W = FV.questions[i], FV.units[i], FV.answers[i], FV.FB_good[i], FV.FB_wrong[i]
        id = i+1 
        question_widget = pn.widgets.StaticText(value=question, width = 750)
        unit_widget = pn.widgets.StaticText(value=unit, width = 10)
        num_widget = pn.widgets.FloatInput(value=0, step=0.01, width = 100)
        #feedback_widget = pn.widgets.TextInput(value="", name="", width=500)
        feedback_widget = pn.widgets.StaticText(value="", name="", width=500)
        submit_button =  pn.widgets.Button(name="Submit")
        
        Hbox = pn.Row(num_widget, unit_widget, submit_button, feedback_widget)       
        quiz_widget = pn.Column(question_widget, Hbox)

        all_widgets.append(quiz_widget)

        # the values for the submit button are determined at the moment these are created.
        attempt = pn.widgets.FloatInput(value=0)
        attempts.append(attempt)

        FV2 = classify_variables(locals())
        submit_button.on_click(check_nummeric_answers(FV2, attempt))
        
    return all_widgets


def Q1():
    T1 = round(uniform(5, 8), 1)
    h1 = round(uniform(0.5, 5), 1)

    text_general = "Can you asses the wave length in three different ways? Firstly through an iterative approach. The wave period ($T$) is " + str(T1) + " seconds, and the water depth ($h$) is " + str(h1) + " m?"
    text_widget = pn.widgets.StaticText(value=text_general, width = 750)
    
    Q1_text = "Q1a) What is the deep water wavelength?"
    Q1_unit = " m"
    L = 9.81 * T1**2 / (2 * np.pi)
    Q1_answer = round(L, 2)
    Q1_FB_G = 'Indeed, the deep water wavelength is in this way related to the wave period.'
    Q1_FB_W = 'There is a mistake, the only variable is the wave period.'

    questions = [Q1_text, Q1_text,Q1_text]
    units = [Q1_unit, Q1_unit, Q1_unit]
    answers =[Q1_answer, Q1_answer, Q1_answer]
    FB_good = [Q1_FB_G, Q1_FB_G, Q1_FB_G]
    FB_wrong = [Q1_FB_W, Q1_FB_W, Q1_FB_W]

    FV = classify_variables(locals())
    all_widgets = nummeric_question_body(FV, random_order = True)
    
    display(pn.Column(text_widget,*all_widgets))
    
Q1()

##### Store parameters manually

#### Storing specific parameters (keys), rather than doing it through locals() or globals()

It might be useful to store or add only specific parameters, for this a dictionary has to be made that includes all the predefined parameter names (keys). These keys can then be added to another class with the function add_local_variables_to().

In [ ]:
par1 = 5.2
par2 = 4.2

class class_variables:
    def __setattr__(self, key, value):
        object.__setattr__(self, key, value)

def classify_variables(params, params2 = {}, max_size_MB = 0):
    max_size = max_size_MB * 1024*1024
    FV = class_variables()
    for key, value in {**params,**params2}.items():
        if sys.getsizeof(value) < max_size or max_size <= 0:
            FV.__setattr__(key, value)
    return FV

def add_local_variables_to(FV,params, max_size_MB = 0):
    max_size = max_size_MB * 1024*1024
    for key, value in  params.items():
        if sys.getsizeof(value) < max_size or max_size <= 0:
            FV.__setattr__(key, value)
    return FV

def build_global_dictionary(keys):
    return {key: globals()[key] for key in keys}

keys = ['par1', 'par2']
params_dict = build_global_dictionary(keys)
GV = classify_variables(params_dict)

print(GV.par1, GV.par2)

The same can be done for locals(). These have to be given to the function to work properly.

In [ ]:
def build_local_dictionary(keys, locals):
    return {key: locals[key] for key in keys}

def function():
    par1_local = 6
    par2_local = 7

    keys = ['par1_local']
    params_dict = build_local_dictionary(keys, locals())
    FV = classify_variables(params_dict)

    print(FV.par1_local)

function()

### Widget calculator for students

Students have to do calculations. On the one hand, we want to offer a way that coding (skill) is not required, on the other hand, it brings very useful insights. Some of the calculations are also repetitive, and doing the calculation multiple times is for the students not useful (especially if they do it by hand or an iterative calculation in Excel, for example). One solution can be to give students a tool that does the calculation for them. So students have to give the input parameters and they immediately get the answer. The request for students is to do the calculation themself once. They can compare their answer with the tool or an answer to a specific question. <br>

One tool that can be used is the param component (https://panel.holoviz.org/reference/panes/Param.html)
It is for now made with the same widgets as it is done before.

The functions that are used to calculate the answers can be called in this tool. The function of calculating the wavelength will be used in the example below.

#### Display wave length calculator

In [ ]:
def wave_length(T, h):
    L = 9.81 * T**2 / (2 * np.pi)
    L_all = [L]

    for i in range(1500):
        L = 9.81 * T**2 / (2 * np.pi) * np.tanh(2 * np.pi * h / L)
        L_all.append(L)

        if np.abs(L_all[-1] - L_all[-2]) < 0.0005:
            break

    return round(L, 13)

In [ ]:
def wavelength_calculator():
    # define widgets (with initial value for L)
    T = pn.widgets.FloatInput(value=5, step=0.01, width=100)
    h = pn.widgets.FloatInput(value=3, step=0.01, width=100)
    
    L_init = wave_length(T.value, h.value)
    L = pn.widgets.FloatInput(value=L_init, step=0.001, width= 100, disabled=True)

    # change the value of L when a change in T or h is observed
    def update_L_widget(event):
        L.value = wave_length(T.value, h.value)

    T.param.watch(update_L_widget, 'value')
    h.param.watch(update_L_widget, 'value')

    # set the surrounding layout, like headings and descriptions
    ## set headings and the column width
    title_input = pn.widgets.StaticText(value='<b>Input</b>', width=200)
    title_output = pn.widgets.StaticText(value='<b>Output</b>', width=200)
    heading = pn.widgets.StaticText(value='<span style="font-size: 20px;"><b>Wavelength calculator</b></span>')

    ## Add descriptions and width for alignment
    symbol_T = pn.widgets.StaticText(value='T', width=1)
    unit_T =  pn.widgets.StaticText(value='s', width=1)
    T_widget = pn.Row(symbol_T, T, unit_T)

    symbol_h = pn.widgets.StaticText(value='h', width = 1)
    unit_h =  pn.widgets.StaticText(value='m', width = 1)
    h_widget = pn.Row(symbol_h, h, unit_h)

    symbol_L = pn.widgets.StaticText(value='L', width = 1)
    unit_L =  pn.widgets.StaticText(value='m', width = 1)
    L_widget = pn.Row(symbol_L, L, unit_L)

    # add latex formula
    text_formula = pn.widgets.StaticText(value='Solves iteratively:', width = 100)
    formula = pn.pane.LaTeX(r"$L=\frac{gT^2}{2 \pi} tanh( \frac{2 \pi h}{L})$")
    row_formula = pn.Row(text_formula, formula)

    ## merge the layout and display the result
    input_widget = pn.Column(title_input, T_widget, h_widget)
    output_widget = pn.Column(title_output, L_widget)
    horizontal_allignment = pn.Row(input_widget, output_widget)
    include_heading = pn.Column(heading,row_formula, horizontal_allignment)

    display(include_heading)

wavelength_calculator()

### Allign multiple calculators (or widgets)

It is possible to structure various widgets next to each other when the functions return the widgets rather than only displaying. A function directly displays a widget if it is called, as is demonstrated in the code below. The widgets can be structured through pn.Row , pn.Column, and pn.Tabs (and more).

#### return rather than dislay

In [ ]:
def wavelength_calculator():
    # define widgets (with initial value for L)
    T = pn.widgets.FloatInput(value=5, step=0.01, width=100)
    h = pn.widgets.FloatInput(value=3, step=0.01, width=100)
    
    L_init = wave_length(T.value, h.value)
    L = pn.widgets.FloatInput(value=L_init, step=0.001, width= 100, disabled=True)

    # change the value of L when a change in T or h is observed
    def update_L_widget(event):
        L.value = wave_length(T.value, h.value)

    T.param.watch(update_L_widget, 'value')
    h.param.watch(update_L_widget, 'value')

    # set the surrounding layout, like headings and descriptions
    ## set headings and the column width
    title_input = pn.widgets.StaticText(value='<b>Input</b>', width=200)
    title_output = pn.widgets.StaticText(value='<b>Output</b>', width=200)
    heading = pn.widgets.StaticText(value='<span style="font-size: 20px;"><b>Wavelength calculator</b></span>')

    ## Add descriptions and width for alignment
    symbol_T = pn.widgets.StaticText(value='T', width=1)
    unit_T =  pn.widgets.StaticText(value='s', width=1)
    T_widget = pn.Row(symbol_T, T, unit_T)

    symbol_h = pn.widgets.StaticText(value='h', width = 1)
    unit_h =  pn.widgets.StaticText(value='m', width = 1)
    h_widget = pn.Row(symbol_h, h, unit_h)

    symbol_L = pn.widgets.StaticText(value='L', width = 1)
    unit_L =  pn.widgets.StaticText(value='m', width = 1)
    L_widget = pn.Row(symbol_L, L, unit_L)

    # add latex formula
    text_formula = pn.widgets.StaticText(value='Solves iteratively:', width = 100)
    formula = pn.pane.LaTeX(r"$L=\frac{gT^2}{2 \pi} tanh( \frac{2 \pi h}{L})$")
    row_formula = pn.Row(text_formula, formula)

    ## merge the layout and display the result
    input_widget = pn.Column(title_input, T_widget, h_widget)
    output_widget = pn.Column(title_output, L_widget)
    horizontal_allignment = pn.Row(input_widget, output_widget)
    include_heading = pn.Column(heading,row_formula, horizontal_allignment)

    return include_heading

wavelength_calculator()

#### Allign multiple calculator tools

The wavelength_calculator is placed in the same row and then displayed, so the widgets are horizontally alligined.

In [ ]:
def calculators():
    allignment = pn.Row(wavelength_calculator(), wavelength_calculator())
    display(allignment)

calculators()

#### place widget in tabs

The widgets can be structured in various ways. <br>
https://panel.holoviz.org/reference/index.html#layouts

One possible way is through Tabs. <br>
https://panel.holoviz.org/reference/layouts/Tabs.html

Here is the same widget displayed in 2 different tabs.

In [ ]:
tabs = pn.Tabs(('wave length calulator', wavelength_calculator()), ('other calculator', wavelength_calculator()))
display(tabs)

## Prestudie results

A study of the implementation of IPY widgets has been done, and after that, a switch to the widgets of panel has been made. The structure is very similar, although some different coding challenges are occurring. The results are shown below. Some uninstalled (or expired versions) packages might be required, therefore is the code placed as raw text.

### A single question

In [ ]:
# the information of the question
question_1 = "A large continental shelf width is at a:"
choices_1 = ["Leading edge", "Trailing edge", "marginal sea"]
answer_1 = choices_1[1]
hint_1 = "Unfortunately not, here is a hint ..."
comment_1 = "Indeed, .. some additional information ... "

# make the required widgets
question_widget = ipw.Label(value=question_1)
choices_widget = ipw.Dropdown(options=choices_1, description="", disabled=False)
submit_button = ipw.Button(description="Submit")
feedback_widget = ipw.Text(value="", placeholder="", description="", disabled=False,layout=ipw.Layout(width="500px"))

# align the widgets vertically
quiz_widget = ipw.VBox([question_widget] + [choices_widget] + [submit_button] + [feedback_widget])

# display the widgets
display(quiz_widget)

def check_answers(button):
    chosen_answer = choices_widget.value
    correct_answer = answer_1

    if chosen_answer == correct_answer:
        feedback_widget.value = comment_1
    else:
        feedback_widget.value = hint_1

# Run the function check_answers when the submit button is pressed
submit_button.on_click(check_answers)

### Multiple questions

In [ ]:
# the information of the questions
question_1 = "A large continental shelf width is at a:"
choices_1 = ["Leading edge", "Trailing edge", "marginal sea"]
answer_1 = choices_1[1]
hint_1 = "Unfortunately not, here is a hint ..."
comment_1 = "Indeed, .. some additional information ... "

question_2 = "The oceanic geoid is: "
choices_2 = ["An oval shape", "The shape of the ocean surface under only gravity forces", "A geo triangle with a different shape", "The actual ocean surface"]
answer_2 = choices_2[1]
hint_2 = "Unfortunately not, here is a hint ..."
comment_2 = "Indeed, .. some additional information ... "

# store the questions in a list
questions = [question_1, question_2]
choices = [choices_1, choices_2]
answers = [answer_1, answer_2]
hints = [hint_1, hint_2]
comments = [comment_1, comment_2]

# an empty list to store the widgets
all_widgets = []  # for visualization, store all the widgets in the order they are going to be displayed
question_widgets = []  # store all the question widgets in a list
choices_widgets = []  # store all the choices widgets in a list

# make the widgets in a loop, one widget states the question and one shows the options that can be selected.
for question, choice, answer, hint, comment in zip(questions, choices, answers, hints, comments):
    question_widget = ipw.Label(value=question,layout=ipw.Layout(width="300px"))
    choices_widget = ipw.Dropdown(
        options=choice, description="Choices:", disabled=False, layout=ipw.Layout(width="300px"))

    all_widgets.append(question_widget)
    all_widgets.append(choices_widget)
    question_widgets.append(question_widget)
    choices_widgets.append(choices_widget)

# make a submit button and a feedback button
submit_button = ipw.Button(description="Submit")
feedback_widget = ipw.Text(value="", placeholder="",description="Feedback:",disabled=False, layout=ipw.Layout(width="500px"))

# allign the submit button and the feedback widget horizontally
HBox_check = ipw.HBox([submit_button, feedback_widget])

# allign the widgets vertically and display them.
Vbox = ipw.VBox(all_widgets + [HBox_check])
display(Vbox)


# make a function to calculate the score and to give feedback
def check_answers(button):
    score = 0

    for i in range(len(questions)):
        answer = choices_widgets[i].value
        correct_answer = answers[i]

        if answer == correct_answer:
            score += 1

    feedback_widget.value = "Your score is " + str(score) + "/" + str(len(questions))


submit_button.on_click(check_answers)

### Multiple selection

In [ ]:
correct_statements = ["Correct", "The earth is round", "Good"]
false_statements = ["False", "The earth is a cube"]

# Make empty list to store the widgets (refences), checkbox and true/false statements sorted.
check_boxes = []  # all the boxes to click
all_statements = []  # all the statements

# An empty list for visualization to store the Hboxes that contains the widgets, one statement and the corresponding checkbox
all_widgets = []

for statement in correct_statements + false_statements:
    add_statement = ipw.Label(value=statement, layout=ipw.Layout(width="150px"))
    check_box_widget = ipw.Checkbox(value=False, description="", layout=ipw.Layout(width="120px"))
    HBox1 = ipw.HBox([add_statement] + [check_box_widget])

    all_statements.append(add_statement)
    check_boxes.append(check_box_widget)
    all_widgets.append(HBox1)

# randomize the order of statements
shuffle(all_widgets)

# add submit button and output with, which come on the bottom 
submit_button = ipw.Button(description='Submit')
output_widget = ipw.Text(value= '', placeholder='', description='', disabled=False)

# make an additional Hbox for alligning the submit button and the output widget
HBox2 = ipw.HBox([submit_button] + [output_widget])
all_widgets.append(HBox2)

# allign all the Hboxes beneath each other (oldest below if not randomized) and display them.
quiz_widget = ipw.VBox(all_widgets)
display(quiz_widget)

# Check the checkbox for each statement and calculate the score.
def check_answers(button):
    score = 0

    for i in range(len(check_boxes)):    
        check_box = check_boxes[i]
        statement = all_statements[i].value

        if statement in correct_statements:
            
            if check_box.value == True:
                score += 1  
                #print("Checkbox is checked for: ", statement, '+1=', score)
            else:
                score -= 0
                #print("Checkbox is checked for: ", statement, '-0=', score)
                
        if statement not in correct_statements:
            if check_box.value == True:
                score -= 1
                #print("Checkbox is unchecked for: ", statement, '-1=', score)
                
            else:
                score -= 0
                #print("Checkbox is unchecked for: ", statement, '+0=', score)
    
    score = np.max([score, 0])
    output_widget.value = str('Your final score is:' +  str(score))
    #print('Your final score is:', score)

submit_button.on_click(check_answers)

### Multiple Choice questions using dictonaries.

In [ ]:
import ipywidgets as widgets # the same as ipw

Inspired by: <br>
https://ipywidgets.readthedocs.io/en/7.6.2/examples/Widget%20List.html <br>
https://www.makeuseof.com/python-make-interactive-quiz-game/

In [ ]:
# with widget a rule usually has to end with a ',' to prevent errors

# Multiple-choice questions with answers, hints, and commentary. In the function each part between {} is assesed independently in a loop.
single_question = [
    {
        'question': 'How many answers are availabe in this question?',
        'choices': ['One', 'Two', 'Three', 'Four'],
        'answer': 'Four',
        'hint' : 'This is unfortunate, please count the answers again.',
        'comment': 'You passed the first question.'
    }
]

First_questions = [
    {
        'question': 'A large continental shelf width is at a',
        'choices': ['Leading edge', 'Trailing edge', 'marginal sea'],
        'answer': 'Trailing edge',
        'hint': 'The continental shelf must not sink',
        'comment': 'Indeed, a large continental shelf width does not occur at a leading edge'
    },
    {
        'question': 'The oceanic geoid is',
        'choices': ['An oval shape', 'The shape of the ocean surface under only gravity forces', 'A geotriangle with a different shape', 'The actual ocean surface'],
        'answer': 'The shape of the ocean surface under only gravity forces',
        'hint': 'This is not the definition of a geoid',
        'comment': 'Indeed, gravitation by landmasses and ice influence the shape of the ocean surface'
    }
]# these questions are asked/tested a few cells down

# Create a quiz, with widget for the answers (choices), a submit button which checks the given answer (widget input value) and gives responses
def multiple_choice_quiz(questions):
    # Make a widget for each question
    question_widgets = [] # a list of all the questions, alternating question (label) with corresponding choices (Dropdown). (If two different list are made then it will first display all the questions and then all the answers)
    for question in questions:
        question_widget = widgets.Label(value=question['question'],)
        choices_widget = widgets.Dropdown(options=question['choices'], description='Choices:', disabled=False,)
        
        question_widgets.append(question_widget)
        question_widgets.append(choices_widget)

    # Create a submit button widget
    submit_button = widgets.Button(description='Submit',)

    # Combine the question widgets and the submit button into a vertical box
    quiz_widget = widgets.VBox(question_widgets + [submit_button])
    display(quiz_widget)    
    
    # Check the answers when the submit button is clicked, which is called below. 
    def check_answers(button):
        correct_answers = 0
        print('')
        
        # check if the answers are answered correctly or wrong and then gives a response
        for i in range(len(questions)):
            if question_widgets[2*i+1].value == questions[i]['answer']:# answer is good
                correct_answers += 1
                              
                if len(questions[i]['comment']) > 0: # commentary is typed
                    print(questions[i]['comment'])
            
            if question_widgets[2*i+1].value != questions[i]['answer'] and len(questions[i]['hint']) > 0: #answer is wrong and a hint can be given
                print(questions[i]['hint'])
                    
        print('You got', correct_answers, 'out of', len(questions), 'questions correct.')

    # Run function check_answers when the submit button is pressed
    submit_button.on_click(check_answers)

    return quiz_widget

In [ ]:
quiz_widget = multiple_choice_quiz(single_question)

In [ ]:
quiz_widget = multiple_choice_quiz(First_questions)

### Multiple choice questions using dictoniaries and xarray

A nice option would be to select a specific question. So all the questions can be described in one dictionary, and then specific ones are asked. This gives the opportunity to have a large question pool from which a few are presented, randomizing the knowledge questions. <br>

The function has worked in the past (june 2023), but it now gives an error, it goes wrong in building the xarray (something has changed, and could not be fixed fast). The code is now as raw text.

### Select true using dictonaries and xarray

With a similar structure as before, now having good and wrong statements. Also here goes something wrong in building the xarray, the function has worked in june 2023.

In [ ]:
all_questions = [
    {
        'id': '1A',
        'question': 'Which statements are True?',
        'correct': ['This is correct', 'Good'],
        'false': ['This is false', 'Wrong', 'Not good'],
        'hint': '' ,
        'comment': '',
        'figure' : ''
    },
    {
        'id': '1B',
        'question': 'Which statements are True?',
        'correct': ['The earth is round'],
        'false': ['The earth is flat', 'The earth is a square'],
        'hint': '-',
        'comment': '-',
        'figure' : ''# [Path.joinpath(path_figures, "Test_figure.png")]
    }
]

And one improved version, also not working due to the xarray.

### Wave components progating over time

#### Plot one wave component

Usefull links to make vboxes: <br>
https://ipywidgets.readthedocs.io/en/7.6.2/examples/Widget%20List.html <br>
https://kapernikov.com/ipywidgets-with-matplotlib/ <br>
https://stackoverflow.com/questions/58416763/change-the-alignment-of-the-content-of-a-label-in-ipywidget <br>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import ipywidgets as widgets

In [ ]:
# required to have animation displayed
%matplotlib widget

#### Plot 3 wave components with amplitude, wave number and radial frequency as input parameters

#### Plot 3 wave components with amplitude, wave period, and length as input parameters

#### Ask question where the answer is based on the input on the graph

The function above can be used as input for questions. 